In [6]:
# ======================
# Bibliotecas padrão para utilidades básicas e manipulação de arquivos:
# ======================
import os
import sys
import time
import zipfile
import glob
from datetime import datetime, timedelta

# ======================
# Registros de logs:
# ======================
import logging

# ======================
# Requisições e manipulação de conteúdo web:
# ======================
import requests
from bs4 import BeautifulSoup

# ======================
# Utilidades e manipulação de dados:
# ======================
from collections import Counter
import chardet
import string
import pandas as pd
import gzip
import shutil
#import pycep_correios  # Descomente se necessário
import brazilcep

# ======================
# Manipulação de datas:
# ======================
from dateutil.relativedelta import relativedelta

# ======================
# Criptografia:
# ======================
import secrets
import base64
#from Crypto.Cipher import AES
#from Crypto.Random import get_random_bytes

# ======================
# Multitarefas:
# ======================
from concurrent.futures import ThreadPoolExecutor

# ======================
# Spark:
# ======================
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import (
    regexp_replace,
    when,
    length,
    to_date,
    upper,
    lower,
    col,
    udf,
    split,
    explode,
    coalesce,
    concat_ws,
    concat,
    lit,
    broadcast,
    regexp_extract,
    expr
)
import pyspark.sql.functions as F
from functools import reduce

# ======================
# Geolocalização:
# ======================
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [7]:
import findspark
findspark.init()

In [8]:
#Bibliotecas padrão: Essas são bibliotecas padrão do Python para operações gerais e manipulação de arquivos.

#Registros de logs: Para fazer registros de logs.

#Requisições e manipulação de conteúdo web: Usado para fazer requisições web e manipular conteúdo HTML/XML.

#Utilidades e manipulação de dados: Conjunto diversificado de bibliotecas para manipulação de dados, codificação de arquivos e outras utilidades.

#Manipulação de datas: Para cálculos e manipulações relacionados a datas.

#Criptografia: Utilitários e bibliotecas para lidar com criptografia.

#Multitarefas: Permite a execução paralela de tarefas.

#Spark: Todas as importações relacionadas ao PySpark.

#Geolocalização: Bibliotecas para geolocalização e geocodificação.

In [4]:
print(os.environ.get("SPARK_HOME"))
print(os.environ.get("HADOOP_HOME"))
print(os.environ.get("JAVA_HOME"))
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"

C:\Users\pedro\spark-3.5.0-bin-hadoop3
C:\Users\pedro\hadoop3.0
C:\Program Files\Java\jdk1.8.0_202


# Configuração do Spark

In [5]:
# RAM: 32GB
# CPU: 6 núcleos (12 threads)
# Storage: SSD de 1TB

spark = (SparkSession.builder
    .master("local[*]")  # Utilizar todos os 12 threads disponíveis
    .config("spark.driver.cores", "3")  # Alocar metade dos núcleos para o driver
    .config("spark.driver.memory", "12g")  # Alocar 12GB para a memória do driver
    .config("spark.default.parallelism", "32")  # Paralelismo padrão baseado no número de threads
    .config("spark.executor.cores", "2")  # Como está em modo local, o executor pode usar metade dos núcleos
    .config("spark.executor.instances", "3")  # Em modo local, você geralmente tem apenas 1 instância de executor
    .config("spark.executor.memory", "6g")  # Alocar 16GB para a memória do executor
    .config("spark.memory.fraction", "0.8")  # Fraction da heap memory a ser usada para armazenamento e cache
    .config("spark.memory.storageFraction", "0.5")  # Fraction da memória de armazenamento que é reservada como memória não imune a evicção
    .config("spark.memory.offHeap.enabled", "true")  # Habilitar memória off-heap
    .config("spark.memory.offHeap.size", "2g")  # Alocar 4GB para off-heap
    .config("spark.executor.memoryOverhead", "1g")  # Overhead de memória fora do heap para o executor. Se não for definido, Spark calculará um valor padrão
    .config('spark.sql.repl.eagerEval.enabled', True)
    .config("spark.sql.repl.eagerEval.maxNumRows", 10)
    .config("spark.shuffle.compress", "true")   
    .config("spark.storage.level", "MEMORY_AND_DISK")   
    .config("spark.rdd.compress", "true")   
    .appName('dataset_cnpj')
    .getOrCreate())


In [5]:
spark = (SparkSession.builder
    .master("local[8]")  # Use all 8 threads.
    .config("spark.driver.cores", "3")  # Use half the threads for the driver.
    .config("spark.driver.memory", "8g")  # Allocate 12GB to the driver to leave room for the OS and other processes.
    .config("spark.default.parallelism", "16")  # Default parallelism, you can adjust based on your data and tasks. Double the thread count is a good start.
    .config("spark.executor.cores", "2")  # Use 4 cores per executor.
    .config("spark.executor.instances", "1")  # Since it's local mode, only one executor instance.
    .config("spark.executor.memory", "4g")  # Assign 8GB for executor memory. Adjust based on your needs.
    .config("spark.memory.fraction", "0.7")  # Fraction of (heap space - 300MB) used for execution and storage. Adjust if needed.
    .config("spark.memory.storageFraction", "0.5")  # Amount of storage memory immune to eviction, expressed as a fraction of the size of the region set aside by spark.memory.fraction.
    .config("spark.memory.offHeap.enabled", "true")  
    .config("spark.memory.offHeap.size", "1g")  # 3GB off-heap memory.
    .config("spark.executor.memoryOverhead", "1g")  # Overhead memory. You might need to adjust depending on your tasks.
    .config('spark.sql.repl.eagerEval.enabled', True)
    .config("spark.sql.repl.eagerEval.maxNumRows", 10)
    .appName('dataset_cnpj')
    .getOrCreate())

In [7]:
spark

In [6]:
pip install python-magic

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
pip install python-magic-bin

     -------------------------------------- 409.3/409.3 kB 8.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install chardet
pip install python-magic
pip install python-magic-bin

SyntaxError: invalid syntax (Temp/ipykernel_22072/4068691271.py, line 1)

# Classe extração receita

In [7]:
from functools import reduce
import os
import chardet
import logging
import glob
import secrets
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import regexp_replace, when,length,to_date,upper,lower,col,split,explode,coalesce,concat_ws,concat,lit,broadcast,regexp_extract,month,year,to_date
from pyspark.sql.functions import broadcast,expr,udf
#from geopy.geocoders import Nominatim
#from geopy.extra.rate_limiter import RateLimiter

   
def geocode_address(address):
    """
    Geocodifica um endereço, convertendo-o em coordenadas de latitude e longitude.

    Parâmetros:
        address (str): Endereço a ser geocodificado.

    Retorna:
        tuple: Um par contendo a latitude e a longitude do endereço fornecido. 
                Se o endereço não puder ser geocodificado, retorna (None, None).

    Exemplo:
        lat, lon = geocode_address("1600 Amphitheatre Parkway, Mountain View, CA")

    Notas:
        - Usa o serviço Nominatim para a geocodificação.
        - Incorpora um limitador de taxa para garantir que não excedamos os limites de requisições por segundo 
            do serviço.
    """
    geolocator = Nominatim(user_agent="CNPJ_GEOLOCATION")
    geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
    location = geocode(address)
    if location:
        return (location.latitude, location.longitude)
    else:
        return (None, None)


class ReceitaLT:
    """
    A classe `ReceitaLT` facilita a manipulação e análise de dados da Receita Federal do Brasil.

    Atributos:
        spark (SparkSession): Sessão Spark para manipulação de dataframes.
        logger (Logger): Logger para capturar e exibir logs.
        
    Atributos estáticos:
        - estabelecimentos: Schema para dados de estabelecimentos.
        - empresas: Schema para dados das empresas.
        - municipios: Schema para municípios.
        - cnaes: Schema para CNAEs.
        - paises: Schema para países.
        - qualificacoes: Schema para qualificações.
        - socios: Schema para sócios.
        - simples: Schema para opções do Simples Nacional.
        - naturezas: Schema para naturezas jurídicas.
        - motivos: Schema para motivos de situações cadastrais.
        - dic_provedor: Dicionário para correção de nomes de provedores de email.
        
    Métodos:
        detect_encoding(file_pattern_or_path, num_bytes=10000): Detecta a codificação do arquivo ou arquivos fornecidos.

    Uso:
        1. Instancie a classe com uma sessão Spark.
        2. Utilize os schemas estáticos para leitura de arquivos.
        3. Use o método `detect_encoding` para determinar a codificação de arquivos antes de lê-los.
        
    Exemplo:
        from pyspark.sql import SparkSession
        
        spark_session = SparkSession.builder.appName("MyApp").getOrCreate()
        receita_helper = ReceitaLT(spark_session)
        encodings = receita_helper.detect_encoding("path/to/datafile.csv")
        df = spark_session.read.csv("path/to/datafile.csv", schema=ReceitaLT.empresas, encoding=encodings["path/to/datafile.csv"])
    """
    def __init__(self, spark: SparkSession):
        """
        Inicializa a classe ReceitaLT.
        
        Parâmetros:
        spark (SparkSession): Uma sessão Spark ativa.
        """
        self.spark = spark
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO) 
        
    # Definindo os schemas:
    estabelecimentos = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("CNPJ_ORDEM", StringType(), nullable=True),
        StructField("CNPJ_DV", StringType(), nullable=True),
        StructField("MATRIZ_FILIAL", StringType(), nullable=True),
        StructField("NOME_FANTASIA", StringType(), nullable=True),
        StructField("SIT_CADASTRAL", IntegerType(), nullable=True),
        StructField("DT_SIT_CADASTRAL", StringType(), nullable=True),
        StructField("MOTIVO_CADASTRAL", StringType(), nullable=True),
        StructField("NOME_CIDADE_EXTERIOR", StringType(), nullable=True),
        StructField("PAIS", StringType(), nullable=True),
        StructField("DT_INICIO_ATIVIDADE", StringType(), nullable=True),
        StructField("CNAE_1", StringType(), nullable=True),
        StructField("CNAE_2", StringType(), nullable=True),
        StructField("TIPO_LOUGRADOURO", StringType(), nullable=True),
        StructField("LOGRADOURO", StringType(), nullable=True),
        StructField("NUMERO", IntegerType(), nullable=True),
        StructField("COMPLEMENTO", StringType(), nullable=True),
        StructField("BAIRRO", StringType(), nullable=True),
        StructField("CEP", IntegerType(), nullable=True),
        StructField("UF", StringType(), nullable=True),
        StructField("MUNICIPIO", StringType(), nullable=True),
        StructField("DDD1", StringType(), nullable=True),
        StructField("TEL1", StringType(), nullable=True),
        StructField("DDD2", StringType(), nullable=True),
        StructField("TEL2", StringType(), nullable=True),
        StructField("DDD_FAX", IntegerType(), nullable=True),
        StructField("FAX", IntegerType(), nullable=True),
        StructField("EMAIL", StringType(), nullable=True),
        StructField("SIT_ESPECIAL", StringType(), nullable=True),
        StructField("DT_SIT_ESPECIAL", StringType(), nullable=True)])

    empresas = StructType([
        StructField("CNPJ", StringType(), nullable=True),
        StructField("NOME_EMPRESA", StringType(), nullable=True),
        StructField("COD_NAT_JURICA", StringType(), nullable=True),
        StructField("QUALIF_RESPONVAVEL", StringType(), nullable=True),
        StructField("CAP_SOCIAL", StringType(), nullable=True),
        StructField("PORTE", StringType(), nullable=True),
        StructField("ENTE_FEDERATIVO", StringType(), nullable=True)])

    municipios = StructType([
        StructField("ID_MUNICPIO", StringType(), nullable=True),
        StructField("MUNICIPIO", StringType(), nullable=True)])

    cnaes = StructType([
        StructField("COD_CNAE", StringType(), nullable=True),
        StructField("CNAE", StringType(), nullable=True)])
    
    paises = StructType([
        StructField("COD_PAIS", StringType(), nullable=True),
        StructField("NM_PAIS", StringType(), nullable=True)])
    
    qualificacoes = StructType([
        StructField("COD_QUALIFICACAO", StringType(), nullable=True),
        StructField("NM_QUALIFICACAO", StringType(), nullable=True)])

    socios = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("IDENTIFICADOR_SOCIO", IntegerType(), nullable=True),
        StructField("NOME_SOCIO_RAZAO_SOCIAL", StringType(), nullable=True),
        StructField("CNPJ_CPF_SOCIO", StringType(), nullable=True),
        StructField("QUALIFICAÇAO_SOCIO", StringType(), nullable=True),
        StructField("DATA_ENTRADA_SOCIEDADE", StringType(), nullable=True),
        StructField("PAIS", StringType(), nullable=True),
        StructField("REPRESENTANTE_LEGAL", StringType(), nullable=True),
        StructField("NOME_REPRESENTANTE", StringType(), nullable=True),
        StructField("QUALIFICACAO_REPRESENTANTE_LEGAL", StringType(), nullable=True),
        StructField("FAIXA_ETARIA", StringType(), nullable=True)])

    simples = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("OPCAO_PELO_SIMPLES", StringType(), nullable=True),
        StructField("DATA_OPCAO_PELO_SIMPLES", StringType(), nullable=True),
        StructField("DATA_EXCLUSAO_SIMPLES", StringType(), nullable=True),
        StructField("OPÇAO_PELO_MEI", StringType(), nullable=True),
        StructField("DATA_OPCAO_PELO_MEI", StringType(), nullable=True),
        StructField("DATA_EXCLUSAO_MEI", StringType(), nullable=True)])

    naturezas = StructType([
        StructField("COD_NAT_JURICA", StringType(), nullable=True),
        StructField("NAT_JURICA", StringType(), nullable=True)])
    
    motivos = StructType([
        StructField("COD_MOTIVO", StringType(), nullable=True),
        StructField("NM_MOTIVO", StringType(), nullable=True)])
    
    dic_provedor = {'0UTLOOK': 'OUTLOOK', '123GMAIL': 'GMAIL', '12GMAIL': 'GMAIL', '19GMAIL': 'GMAIL', '1HOTMAIL': 'HOTMAIL', 
                    '2010HOTMAIL': 'HOTMAIL', '20GMAIL': 'GMAIL', '23GMAIL': 'GMAIL', '2GMAIL': 'GMAIL', '2HOTMAIL': 'HOTMAIL', 
                    '30GMAIL': 'GMAIL', '7GMAIL': 'GMAIL', 'ADV': 'ADV', 'AGMAIL': 'GMAIL', 'AHOO': 'YAHOO', 'AIL': 'AOL', 'ALUNO': 'ALUNO', 
                    'AOL': 'AOL', 'AUTLOOK': 'OUTLOOK', 'BB': 'BB', 'BOL': 'BOL', 'BOLL': 'BOL', 'BOOL': 'BOL', 'BRTURBO': 'OI', 
                    'CAIXA': 'CAIXA', 'CLICK21': 'CLICK21', 'CLOUD': 'ICLOUD', 'CRECI': 'CRECI', 'EDU': 'EDU', 'EMAIL': 'EMAIL', 
                    'FACEBOOK': 'FACEBOOK', 'FMAIL': 'GMAIL', 'G': 'GMAIL', 'G-MAIL': 'GMAIL', 'GAIL': 'GMAIL', 'GAMAIL': 'GMAIL', 
                    'GAMEIL': 'GMAIL', 'GAMIAL': 'GMAIL', 'GAMIL': 'GMAIL', 'GEMAIL': 'GMAIL', 'GGMAIL': 'GMAIL', 'GHMAIL': 'GMAIL', 
                    'GHOTMAIL': 'HOTMAIL', 'GIMAIL': 'GMAIL', 'GLOBO': 'GLOBO', 'GLOBOMAIL': 'LWMAIL', 'GMA': 'GMAIL', 'GMAAIL': 'GMAIL', 
                    'GMAI': 'GMAIL', 'GMAIAL': 'GMAIL', 'GMAII': 'GMAIL', 'GMAIIL': 'GMAIL', 'GMAIK': 'GMAIL', 'GMAIL': 'GMAIL', 
                    'GMAILC': 'GMAIL', 'GMAILGMAIL': 'GMAIL', 'GMAILL': 'GMAIL', 'GMAILMAIL': 'GMAIL', 'GMAILO': 'GMAIL', 'GMAIM': 'GMAIL', 
                    'GMAIO': 'GMAIL', 'GMAIOL': 'GMAIL', 'GMAIS': 'GMAIL', 'GMAISL': 'GMAIL', 'GMAIUL': 'GMAIL', 'GMAL': 'GMAIL', 
                    'GMALI': 'GMAIL', 'GMAOL': 'GMAIL', 'GMAQIL': 'GMAIL', 'GMASIL': 'GMAIL', 'GMAUIL': 'GMAIL', 'GMAUL': 'GMAIL',
                    'GMEIL': 'GMAIL', 'GMIAL': 'GMAIL', 'GMIL': 'GMAIL', 'GML': 'GMAIL', 'GMMAIL': 'GMAIL', 'GMNAIL': 'GMAIL', 
                    'GMQIL': 'GMAIL', 'GMSIL': 'GMAIL', 'GNAIL': 'GMAIL', 'GNMAIL': 'GMAIL', 'GOMAIL': 'GMAIL', 'GOOGLEMAIL': 'GMAIL',
                    'GOTMAIL': 'HOTMAIL', 'GTMAIL': 'GMAIL', 'H0TMAIL': 'HOTMAIL', 'HAHOO': 'YAHOO', 'HATMAIL': 'HOTMAIL', 'HAYOO': 'YAHOO', 
                    'HGMAIL': 'GMAIL', 'HHOTMAIL': 'HOTMAIL', 'HIOTMAIL': 'HOTMAIL', 'HITMAIL': 'HOTMAIL', 'HJOTMAIL': 'HOTMAIL', 
                    'HMAIL': 'HOTMAIL', 'HOITMAIL': 'HOTMAIL', 'HOLMAIL': 'HOTMAIL', 'HOLTMAIL': 'HOTMAIL', 'HOMAIL': 'HOTMAIL', 
                    'HOMTAIL': 'HOTMAIL', 'HOMTIAL': 'HOTMAIL', 'HOMTMAIL': 'HOTMAIL', 'HOOTMAIL': 'HOTMAIL', 'HOPTMAIL': 'HOTMAIL', 
                    'HORMAIL': 'HOTMAIL', 'HORTMAIL': 'HOTMAIL', 'HOT': 'HOTMAIL', 'HOTAIL': 'HOTMAIL', 'HOTAMAIL': 'HOTMAIL', 
                    'HOTAMIL': 'HOTMAIL', 'HOTEMAIL': 'HOTMAIL', 'HOTGMAIL': 'HOTMAIL', 'HOTIMAIL': 'HOTMAIL', 'HOTIMAL': 'HOTMAIL', 
                    'HOTLMAIL': 'HOTMAIL', 'HOTLOOK': 'OUTLOOK', 'HOTMA': 'HOTMAIL', 'HOTMAAIL': 'HOTMAIL', 'HOTMAI': 'HOTMAIL', 
                    'HOTMAIAL': 'HOTMAIL', 'HOTMAII': 'HOTMAIL', 'HOTMAIIL': 'HOTMAIL', 'HOTMAIL': 'HOTMAIL', 'HOTMAILC': 'HOTMAIL', 
                    'HOTMAILL': 'HOTMAIL', 'HOTMAILO': 'HOTMAIL', 'HOTMAIM': 'HOTMAIL', 'HOTMAIO': 'HOTMAIL', 'HOTMAIOL': 'HOTMAIL', 
                    'HOTMAIUL': 'HOTMAIL', 'HOTMAL': 'HOTMAIL', 'HOTMALI': 'HOTMAIL', 'HOTMAMIL': 'HOTMAIL', 'HOTMAOL': 'HOTMAIL', 
                    'HOTMAQIL': 'HOTMAIL', 'HOTMASIL': 'HOTMAIL', 'HOTMAUIL': 'HOTMAIL', 'HOTMAUL': 'HOTMAIL', 'HOTMEIL': 'HOTMAIL', 
                    'HOTMIAIL': 'HOTMAIL', 'HOTMIAL': 'HOTMAIL', 'HOTMIL': 'HOTMAIL', 'HOTMMAIL': 'HOTMAIL', 'HOTMNAIL': 'HOTMAIL',
                    'HOTMQIL': 'HOTMAIL', 'HOTMSIL': 'HOTMAIL', 'HOTNAIL': 'HOTMAIL', 'HOTOMAIL': 'HOTMAIL', 'HOTRMAIL': 'HOTMAIL', 
                    'HOTTMAIL': 'HOTMAIL', 'HOTYMAIL': 'HOTMAIL', 'HOUTLOOK': 'OUTLOOK', 'HOYMAIL': 'HOTMAIL', 'HPTMAIL': 'HOTMAIL', 
                    'HTMAIL': 'HOTMAIL', 'HTOMAIL': 'HOTMAIL', 'HYAHOO': 'YAHOO', 'IAHOO': 'YAHOO', 'IBEST': 'IBEST', 'ICLAUD': 'ICLOUD', 
                    'ICLOD': 'ICLOUD', 'ICLOID': 'ICLOUD', 'ICLOOD': 'ICLOUD', 'ICLOU': 'ICLOUD', 'ICLOUD': 'ICLOUD', 'ICLOUDE': 'ICLOUD', 
                    'ICLOULD': 'ICLOUD', 'ICLOUND': 'ICLOUD', 'ICLUD': 'ICLOUD', 'ICLUOD': 'ICLOUD', 'ICOUD': 'ICLOUD', 'ICOULD': 'ICLOUD', 
                    'ID': 'IG', 'IG': 'IG', 'IGMAIL': 'GMAIL', 'IGUI': 'IG', 'IMAIL': 'GMAIL', 'INCLOUD': 'ICLOUD', 'ITELEFONICA': 'ITELEFONICA',
                    'JMAIL': 'GMAIL', 'JOTMAIL': 'HOTMAIL', 'LIVE': 'LIVE', 'LWMAIL': 'LWMAIL', 'MAIL': 'MAIL', 'ME': 'ME', 'MSM': 'MSN', 
                    'MSN': 'MSN', 'NETSITE': 'NETSITE', 'OI': 'OI', 'OIMAIL': 'HOTMAIL', 'OITLOOK': 'OUTLOOK', 'OLTLOOK': 'OUTLOOK', 
                    'OOUTLOOK': 'OUTLOOK', 'OTLOOK': 'OUTLOOK', 'OTMAIL': 'HOTMAIL', 'OUL': 'UOL', 'OULOOK': 'OUTLOOK', 'OULTLOOK': 'OUTLOOK',
                    'OULTOOK': 'OUTLOOK', 'OUTILOOK': 'OUTLOOK', 'OUTIOOK': 'OUTLOOK', 'OUTLLOK': 'OUTLOOK', 'OUTLLOOK': 'OUTLOOK', 
                    'OUTLOCK': 'OUTLOOK', 'OUTLOK': 'OUTLOOK', 'OUTLOKK': 'OUTLOOK', 'OUTLOOCK': 'OUTLOOK', 'OUTLOOK': 'OUTLOOK', 
                    'OUTLOOKL': 'OUTLOOK', 'OUTLOOL': 'OUTLOOK', 'OUTLOOOK': 'OUTLOOK', 'OUTLUK': 'OUTLOOK', 'OUTOLOOK': 'OUTLOOK',
                    'OUTOOK': 'OUTLOOK', 'OUTOOLK': 'OUTLOOK', 'OUTTLOOK': 'OUTLOOK', 'OUTULOOK': 'OUTLOOK', 'POP': 'POP',
                    'PROTON': 'PROTONMAIL', 'PROTONMAIL': 'PROTONMAIL', 'PUTLOOK': 'OUTLOOK', 'R7': 'R7', 'ROCKETMAIL': 'ROCKETMAIL', 
                    'ROCKTMAIL': 'ROCKETMAIL', 'ROTMAIL': 'HOTMAIL', 'SERCOMTEL': 'SERCOMTEL', 'SETELAGOASGML': 'GMAIL', 
                    'SUPERIG': 'SUPERIG', 'TAHOO': 'YAHOO', 'TERRA': 'TERRA', 'TERRRA': 'TERRA', 'TMAIL': 'GMAIL', 
                    'TVGLOBO': 'GLOBO', 'UAHOO': 'YAHOO', 'UAI': 'UAI', 'UFV': 'UFV', 'UNESP': 'UNESP', 'UNOCHAPECO': 'UNOCHAPECO', 
                    'UO': 'UOL', 'UOL': 'UOL', 'UOTLOOK': 'OUTLOOK', 'UPF': 'UPF', 'USP': 'USP', 'UTLOOK': 'OUTLOOK', 'VELOXMAIL': 'VELOXMAIL',
                    'WINDOWSLIVE': 'WINDOWSLIVE', 'YAAHOO': 'YAHOO', 'YAGOO': 'YAHOO', 'YAHAOO': 'YAHOO', 'YAHHO': 'YAHOO', 'YAHHOO': 'YAHOO', 
                    'YAHO': 'YAHOO', 'YAHOO': 'YAHOO', 'YAHOOCOM': 'YAHOO', 'YAHOOL': 'YAHOO', 'YAHOOO': 'YAHOO', 'YAHOOU': 'YAHOO', 
                    'YANHOO': 'YAHOO', 'YAOO': 'YAHOO', 'YAOOL': 'YAHOO', 'YAROO': 'YAHOO', 'YHAOO': 'YAHOO', 'YHOO': 'YAHOO', 'YMAIL': 'YMAIL', 
                    'YOHOO': 'YAHOO', 'YOPMAIL': 'HOTMAIL', 'ZIPMAIL': 'ZIPMAIL', '_HOTMAIL': 'HOTMAIL',     'GMAUL': 'GMAIL','GMALE': 'GMAIL', 
                    'GMAILE': 'GMAIL', 'GMILE': 'GMAIL', 'HOTMEL': 'HOTMAIL', 'HOTMELL': 'HOTMAIL','HOTMEAL': 'HOTMAIL', 'OUTLOKES': 'OUTLOOK', 
                    'OTLOOKS': 'OUTLOOK', 'YAHU': 'YAHOO', 'YOHU': 'YAHOO', 'YAHUO': 'YAHOO', 'YAHEE': 'YAHOO', 'UOLL': 'UOL',
                    'UOOL': 'UOL', 'UULL': 'UOL', 'ICLODUE': 'ICLOUD', 'ICLAWD': 'ICLOUD', 'ROCKEDMAIL': 'ROCKETMAIL', 'ROKETMAIL': 'ROCKETMAIL',
                    'OUTLOKE': 'OUTLOOK', 'OUTLOOCKE': 'OUTLOOK', 'YAAHO': 'YAHOO', 'YAHOOE': 'YAHOO', 'YAHUE': 'YAHOO', 'HOTMILE': 'HOTMAIL', 'HOTMELE': 'HOTMAIL',
                    'FACEBOKE': 'FACEBOOK', 'FACBOOK': 'FACEBOOK', 'FCEBOOK': 'FACEBOOK', 'BOLL': 'BOL', 'BOLLE': 'BOL', 'BULE': 'BOL', 'GLOBOE': 'GLOBO',
                    'GLOBU': 'GLOBO', 'GMILE': 'GMAIL', 'MSNE': 'MSN', 'MSNN': 'MSN', 'ICLOOUD': 'ICLOUD', 'OUTLUKE': 'OUTLOOK', 'OUTLLOKE': 'OUTLOOK',
                    'GMAUL': 'GMAIL','GMALE': 'GMAIL', 'GMAILE': 'GMAIL', 'GMILE': 'GMAIL', 'HOTMEL': 'HOTMAIL', 'HOTMELL': 'HOTMAIL',
                    'HOTMEAL': 'HOTMAIL', 'OUTLOKES': 'OUTLOOK', 'OTLOOKS': 'OUTLOOK', 'YAHU': 'YAHOO', 'YOHU': 'YAHOO', 'YAHUO': 'YAHOO', 
                    'YAHEE': 'YAHOO', 'UOLL': 'UOL', 'UOOL': 'UOL', 'UULL': 'UOL', 'ICLODUE': 'ICLOUD', 'ICLAWD': 'ICLOUD',  'ROCKEDMAIL': 'ROCKETMAIL',
                    'ROKETMAIL': 'ROCKETMAIL', 'OUTLOKE': 'OUTLOOK', 'OUTLOOCKE': 'OUTLOOK', 'YAAHO': 'YAHOO', 'YAHOOE': 'YAHOO', 'YAHUE': 'YAHOO', 
                    'HOTMILE': 'HOTMAIL', 'HOTMELE': 'HOTMAIL', 'FACEBOKE': 'FACEBOOK', 'FACBOOK': 'FACEBOOK', 'FCEBOOK': 'FACEBOOK', 'BOLL': 'BOL',
                    'BOLLE': 'BOL', 'BULE': 'BOL', 'GLOBOE': 'GLOBO',  'GLOBU': 'GLOBO', 'GMILE': 'GMAIL', 'MSNE': 'MSN', 'MSNN': 'MSN', 'ICLOOUD': 'ICLOUD',
                    'OUTLUKE': 'OUTLOOK', 'OUTLLOKE': 'OUTLOOK', 'PROTONMIAL': 'PROTONMAIL',  'PROTONMALE': 'PROTONMAIL', 'PROTOMAIL': 'PROTONMAIL', 
                    'OULOOKCOM': 'OUTLOOK', 'YAHCOM': 'YAHOO',  'YAHOCOM': 'YAHOO','GAMILCOM': 'GMAIL', 'GMALCOM': 'GMAIL',  'HOTMALCOM': 'HOTMAIL',  
                    'HOTMILCOM': 'HOTMAIL', 'HOTMELCOM': 'HOTMAIL', 'ROCKMAIL': 'ROCKETMAIL', 'ROKMAIL': 'ROCKETMAIL', 'TERA': 'TERRA', 'TEERA': 'TERRA', 
                    'FACBOOKCOM': 'FACEBOOK', 'FACEBOOKCOM': 'FACEBOOK', 'ICLOWD': 'ICLOUD', 'ICLOUND': 'ICLOUD', 'UOOLCOM': 'UOL', 'UOLLCOM': 'UOL', 
                    'UOLCOMBR': 'UOL','LIVECOM': 'LIVE', 'LIVECOMBR': 'LIVE', 'GMAICOM': 'GMAIL',  'GMAILCOMBR': 'GMAIL',  'YAHOOBR': 'YAHOO', 
                    'YAHOOOCOMBR': 'YAHOO', 'YAHOOOCOM': 'YAHOO', 'ZIPMAILE': 'ZIPMAIL', 'ZIPMAILL': 'ZIPMAIL',  'IBESTT': 'IBEST', 'IBESTE': 'IBEST'}

    
    @staticmethod
    def detect_encoding(file_pattern_or_path, num_bytes=10000):
        """
        Detecta a codificação do arquivo ou arquivos fornecidos.
        
        Parâmetros:
            file_pattern_or_path (str): Caminho ou padrão do arquivo para detecção.
            num_bytes (int, opcional): Número de bytes para ler para a detecção. Padrão é 10000.
        
        Retorna:
            dict: Dicionário com caminho do arquivo como chave e codificação detectada como valor.
        """
        files = glob.glob(file_pattern_or_path)
        encodings = {}
        for file_path in files:
            with open(file_path, 'rb') as f:
                rawdata = f.read(num_bytes)
                encodings[file_path] = chardet.detect(rawdata)["encoding"]
        return encodings


    def read_data(self, schema_name, base_path=None):
        """
        Lê dados de vários arquivos CSV de acordo com o esquema e caminho base fornecidos, consolidando-os 
        em um único DataFrame do Spark.

        Parâmetros:
            schema_name (str): Nome do esquema a ser usado para a leitura dos arquivos.
                               Deve ser uma das chaves do dicionário `schemas`.

            base_path (str, opcional): Caminho base dos arquivos CSV.
                                       Se não for fornecido, ele tentará buscar da variável de ambiente 'BASE_PATH'.
                                       Caso não encontre, o padrão "./output" será utilizado.

        Retorna:
            DataFrame: DataFrame do Spark contendo os dados consolidados dos arquivos CSV.

        Exceções:
            Pode lançar uma exceção se o arquivo não estiver presente no caminho especificado ou
            se houver problemas de codificação ao ler o arquivo.

        Exemplo:
            receita_helper = ReceitaLT(spark_session)
            df = receita_helper.read_data("estabelecimentos", "/path/to/csv/files")

        Notas:
            - A função primeiro detecta a codificação dos arquivos antes de lê-los para garantir que 
              eles sejam lidos corretamente.
            - A função lida com múltiplos arquivos CSV e os une em um único DataFrame.
            - O formato de arquivo assumido é CSV com delimitador ";", sem cabeçalho e com aspas para delimitar campos.
        """
        schemas = {
            "estabelecimentos": self.estabelecimentos,
            "empresas": self.empresas,
            "municipios": self.municipios,
            "cnaes": self.cnaes,
            "socios": self.socios,
            "simples": self.simples,
            "naturezas": self.naturezas,
            "qualificacoes": self.qualificacoes,
            "motivos": self.motivos,
            "paises": self.paises}

        # Se o base_path não for fornecido, pegar da variável de ambiente ou usar um padrão.
        if not base_path:
            base_path = os.environ.get('BASE_PATH', "./output")

        if schema_name in ['estabelecimentos', 'empresas', 'socios']:
            file_location_pattern = os.path.join(base_path, schema_name.capitalize(), '*.csv')
        else:
            file_location_pattern = os.path.join(base_path, schema_name.capitalize(), f"{schema_name.capitalize()}.csv")

        # Detectar codificações
        encodings = self.detect_encoding(file_location_pattern)
        self.logger.info(f"Detected encodings: {encodings}")

        # Agora, vamos ler cada arquivo com sua codificação correta e armazenar em uma lista de DataFrames
        dfs = []
        for file_location, encoding in encodings.items():
            df = (self.spark.read.format("csv")
                  .option("sep", ";")
                  .option("header", "false")
                  .option('quote', '"')
                  .option("escape", '"')
                  .option("encoding", encoding)
                  .schema(schemas[schema_name])
                  .load(file_location))
            dfs.append(df)

        # Unir todos os DataFrames em um único DataFrame
        if dfs:
            final_df = reduce(lambda a, b: a.union(b), dfs)
        else:
            final_df = self.spark.createDataFrame([], schemas[schema_name])

        return final_df
    

    # Define the UDF
    schema = StructType([
        StructField("latitude", FloatType(), nullable=True),
        StructField("longitude", FloatType(), nullable=True)
    ])
    
    @udf(schema)
    def geocode_udf(address):
        """
        UDF do Spark para geocodificar um endereço dentro de um DataFrame.

        Parâmetros:
            address (str): Endereço a ser geocodificado.

        Retorna:
            dict: Dicionário contendo a 'latitude' e a 'longitude' do endereço fornecido.
                  Se o endereço não puder ser geocodificado, os valores serão None.

        Exemplo:
            df.withColumn("location", geocode_udf(df["address"]))

        Notas:
            - Esta UDF encapsula a função `geocode_address`.
            - Retorna um tipo de dado complexo (Struct) com dois campos: 'latitude' e 'longitude'.
        """
        global geocode_address
        lat, lon = geocode_address(address)
        return {"latitude": lat, "longitude": lon}
    
    def process_estabelecimentos(self, df):
        
        """
        Processa e enriquece o DataFrame de estabelecimentos com informações adicionais e transformações.

        Parâmetros:
            df (DataFrame): DataFrame inicial contendo informações de estabelecimentos.

        Retorna:
            DataFrame: DataFrame processado e enriquecido com novas colunas e informações.

        Descrição:
            - Lê dataframes adicionais relacionados a países, municípios, cnaes e motivos.
            - Realiza renomeações de colunas para facilitar junções.
            - Enriquece o dataframe com informações de motivos, cnaes, municípios e países.
            - Processa colunas de e-mail, separando provedores e corrigindo valores.
            - Converte colunas de data de string para formato de data.
            - Deriva colunas de ano e mês a partir de datas.
            - Processa e deriva novas colunas com base em mapeamentos para situação cadastral e tipo de estabelecimento.
            - Valida endereços de e-mail usando expressões regulares.
            - Combina informações de endereço para formar uma coluna completa de endereço.
            - Utiliza a função de geocodificação para obter coordenadas com base no endereço e, em caso de falha, com base no CEP.
            - Realiza correções na coluna de provedor de e-mail usando um dicionário de mapeamento.

        Notas:
            - Esta função faz uso intensivo das operações de DataFrame do PySpark.
            - Dependências: A função depende de outras funções e UDFs, como 'geocode_udf', bem como de variáveis de instância, como 'dic_provedor'.
        """
        
        df_pais = self.read_data(schema_name='paises')
        df_mun = self.read_data(schema_name='municipios')
        df_cnaes = self.read_data(schema_name='cnaes')
        df_motivos = self.read_data(schema_name='motivos')
        
        df = df.withColumnRenamed("CNAE_1", "COD_CNAE")
        df = df.withColumnRenamed("MUNICIPIO", "ID_MUNICPIO")
        df = df.withColumnRenamed("PAIS", "COD_PAIS")
        df = df.withColumnRenamed("MOTIVO_CADASTRAL", "COD_MOTIVO") 
        

        df = df.join(broadcast(df_motivos), "COD_MOTIVO", "left").drop(df.COD_MOTIVO)
        df = df.join(broadcast(df_cnaes), "COD_CNAE", "left").drop(df.COD_CNAE)
        df = df.join(broadcast(df_mun), "ID_MUNICPIO", "left").drop(df.ID_MUNICPIO)
        df = df.join(broadcast(df_pais), "COD_PAIS", "left").drop(df.COD_PAIS)
        
        dic_provedor = self.dic_provedor
        # Tratamento da coluna provedor
        df = df.withColumn("PROVEDOR",  regexp_extract("EMAIL", "(?<=@)[^.]+(?=\\.)", 0))
        
        # Colocando em caixa alta o provedor
        df = df.withColumn("PROVEDOR", upper(col("PROVEDOR")))
        
        # Colocando em caixa baixa o email
        df = df.withColumn("EMAIL", lower(col("EMAIL")))

        # Convertendo colunas de data
        df = df.withColumn("DT_SIT_CADASTRAL", to_date(col('DT_SIT_CADASTRAL'), "yyyyMMdd"))
        df = df.withColumn("DT_INICIO_ATIVIDADE", to_date(col('DT_INICIO_ATIVIDADE'), "yyyyMMdd"))
        df = df.withColumn("DT_SIT_ESPECIAL", to_date(col('DT_SIT_ESPECIAL'), "yyyyMMdd"))
        
        df = df.withColumn( "ano_cadastro", year('DT_INICIO_ATIVIDADE'))
        df = df.withColumn( "mes_cadastro", month('DT_INICIO_ATIVIDADE'))
        df = df.withColumn( "ano_sit_cadastral", year('DT_SIT_CADASTRAL'))
        df = df.withColumn( "mes_sit_cadastral", month('DT_SIT_CADASTRAL'))
        
        # Defina o dicionário de mapeamento
        mapping = {1: 'NULA',2: 'ATIVA',3: 'SUSPENSA',4: 'INAPTA',8: 'BAIXADA'}
        
        # Use a função 'when' para criar a nova coluna 'NM_SIT_CADASTRAL'
        df = df.withColumn("NM_SIT_CADASTRAL",
                           when(df["SIT_CADASTRAL"].isin(list(mapping.keys())), df["SIT_CADASTRAL"]).otherwise(None))
        
        # Substitua os valores na nova coluna com base no dicionário de mapeamento
        for key, value in mapping.items():
            df = df.withColumn("NM_SIT_CADASTRAL", when(df["SIT_CADASTRAL"] == key, value).otherwise(df["NM_SIT_CADASTRAL"]))
            
        # Use uma expressão regular para validar os endereços de e-mail
        email_pattern = r'^\S+@\S+\.\S+$'  # Padrão simples de endereço de e-mail
        
        # Use a função 'regexp_extract' para extrair endereços de e-mail válidos
        df = df.withColumn("valid_email", regexp_extract(col("EMAIL"), email_pattern, 0))
        
        # Defina o dicionário de mapeamento
        mapping = {1: 'MATRIZ',2: 'FILIAL'}
        
        # Use a função 'when' para criar a nova coluna 'NM_MATRIZ_FILIAL'
        df = df.withColumn("NM_MATRIZ_FILIAL",
                           when(df["MATRIZ_FILIAL"].isin(list(mapping.keys())), df["MATRIZ_FILIAL"]).otherwise(None))
        
        # Substitua os valores na nova coluna com base no dicionário de mapeamento
        for key, value in mapping.items():
            df = df.withColumn("NM_MATRIZ_FILIAL", when(df["MATRIZ_FILIAL"] == key, value).otherwise(df["NM_MATRIZ_FILIAL"]))
            
        # Criando a nova coluna "ENDERECO_COMPLETO"
        df = df.withColumn("ENDERECO_COMPLETO",
                           concat_ws(", ",
                                     concat(df["TIPO_LOUGRADOURO"], lit(" "), df["LOGRADOURO"]),
                                     "NUMERO",concat_ws(" - ", "MUNICIPIO", "UF")))
        
        # Adicione a lógica de geocodificação aqui
        df = df.withColumn("COORDENADAS", ReceitaLT.geocode_udf(df["ENDERECO_COMPLETO"]))
        
        df = df.withColumn("COORDENADAS",
                           when((col("COORDENADAS.latitude").isNull()) & (col("COORDENADAS.longitude").isNull()),
                                ReceitaLT.geocode_udf(df["CEP"])).otherwise(col("COORDENADAS")))
        
        # Correção da coluna provedor
        df = df.replace(dic_provedor, subset=['PROVEDOR'])

        # Transformação das keys e values do dicionário em lowercase
        dic_prov_lower = {k.lower(): str(v).lower() for k, v in dic_provedor.items()}

        # Correção dos provedores na coluna EMAIL
        replace_expr = reduce(
            lambda a, b: regexp_replace(a, rf"\b{b[0]}\b", b[1]),
            dic_prov_lower.items(),
            col("valid_email"))

        df = df.withColumn("valid_email", replace_expr)
        df = df.withColumnRenamed("valid_email", "VALILD_EMAIL")
        
        return df
    
    def process_empresas(self, df):
        """
        Processa e enriquece o DataFrame de empresas com informações adicionais e transformações.

        Parâmetros:
            df (DataFrame): DataFrame inicial contendo informações de empresas.

        Retorna:
            DataFrame: DataFrame processado e enriquecido com novas colunas e informações.

        Descrição:
            - Lê dataframes adicionais relacionados a naturezas jurídicas e qualificações.
            - Realiza renomeação de colunas para facilitar junções.
            - Enriquece o dataframe com informações de naturezas jurídicas e qualificações.
            - Processa a coluna 'NOME_EMPRESA' para extrair informações potenciais de CPF.
            - Deriva uma nova coluna baseada no porte da empresa, usando um mapeamento predefinido.
            - Determina a probabilidade de um valor ser um CPF válido com base em seu comprimento.
            - Criptografa possíveis valores de CPF usando AES e os armazena em uma nova coluna 'CPF_CRIPTOGRAFADO', enquanto remove a coluna original 'CPF'.

        Notas:
            - Esta função faz uso intensivo das operações de DataFrame do PySpark.
            - O valor de criptografia (secret_key) é gerado dinamicamente a cada chamada da função. Portanto, cada execução resultará em valores de 'CPF_CRIPTOGRAFADO' diferentes para os mesmos CPFs.
            - O método AES usado aqui é 'ECB', que não é considerado seguro para muitos casos de uso devido à falta de vetor de inicialização (IV). A utilização deste modo deve ser revista se a segurança for uma preocupação.
        """
        df_nat = self.read_data(schema_name='naturezas')
        df_qual = self.read_data(schema_name='qualificacoes')
        
        df = df.withColumnRenamed("QUALIF_RESPONVAVEL", "COD_QUALIFICACAO")
        
        df = df.join(broadcast(df_nat), "COD_NAT_JURICA", "left").drop(df.COD_NAT_JURICA)
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop(df.COD_QUALIFICACAO)
               
        df = df.withColumn("CPF", regexp_replace("NOME_EMPRESA", "[^0-9]", ""))
        df = df.withColumn("CPF", when(col("CPF") == "", None).otherwise(col("CPF")))
        df = df.withColumn('CPF_LEN', length('CPF'))
        # Defina o dicionário de mapeamento
        mapping = {0: 'NÃO INFORMADO',1: 'MICRO EMPRESA',3: ' EMPRESA DE PEQUENO PORTE',5: 'DEMAIS',8: 'BAIXADA'}
        # Use a função 'when' para criar a nova coluna 'NM_SIT_CADASTRAL'
        df = df.withColumn("NM_PORTE",
                           when(df["PORTE"].isin(list(mapping.keys())), df["PORTE"]).otherwise(None))
        
        # Substitua os valores na nova coluna com base no dicionário de mapeamento
        for key, value in mapping.items():
            df = df.withColumn("NM_PORTE", when(df["PORTE"] == key, value).otherwise(df["NM_PORTE"]))

        df = df.withColumn("PROBABILIDADE_DE_SER_CPF", when(df["CPF_LEN"] == 11, "SIM").otherwise("NAO"))
        
        secret_key = secrets.token_urlsafe(24)
        df = df.withColumn("CPF_CRIPTOGRAFADO", expr(f"base64(aes_encrypt(CPF, '{secret_key}', 'ECB', 'PKCS'))")).drop(df.CPF)
        
        return df
    
    
    def process_simples(self, df):
        """
        Processa o DataFrame relacionado ao regime tributário SIMPLES das empresas.

        Parâmetros:
            df (DataFrame): DataFrame inicial contendo informações relacionadas ao regime tributário SIMPLES.

        Retorna:
            DataFrame: DataFrame processado com colunas de data convertidas e apenas as colunas relevantes selecionadas.

        Descrição:
            - Converte colunas que representam datas do formato "yyyyMMdd" para o tipo data.
            - Seleciona apenas as colunas relevantes para o contexto, que são: 'CNPJ_BASICO', 'OPÇAO_PELO_MEI', 'DT_OPCAO_MEI', 'DT_EXCLUSAO_MEI', 'OPCAO_PELO_SIMPLES', 'DT_OPCAO_SIMPLES', e 'DT_EXCLUSAO_SIMPLES'.

        Notas:
            - Esta função assume que as colunas de data estão no formato "yyyyMMdd" e realiza a conversão para o tipo data.
            - As colunas de datas que são processadas incluem: DATA_OPCAO_PELO_SIMPLES, DATA_EXCLUSAO_SIMPLES, DATA_EXCLUSAO_MEI e DATA_OPCAO_PELO_MEI.
        """
        df = df.withColumn("DT_OPCAO_SIMPLES", to_date,(col('DATA_OPCAO_PELO_SIMPLES'), "yyyyMMdd"))
        df = df.withColumn("DT_EXCLUSAO_SIMPLES", to_date(col('DATA_EXCLUSAO_SIMPLES'), "yyyyMMdd"))
        df = df.withColumn("DT_EXCLUSAO_MEI", to_date(col('DATA_EXCLUSAO_MEI'), "yyyyMMdd"))
        df = df.withColumn("DT_OPCAO_MEI", to_date(col('DATA_OPCAO_PELO_MEI'), "yyyyMMdd"))
        df = df.select('CNPJ_BASICO','OPÇAO_PELO_MEI','DT_OPCAO_MEI','DT_EXCLUSAO_MEI','OPCAO_PELO_SIMPLES','DT_OPCAO_SIMPLES','DT_EXCLUSAO_SIMPLES')
        
        return df
    
    
    def save_data(self, df, path, num_partitions=1, file_format="parquet"):
        """
        Save the DataFrame to the specified path.
        
        :param df: DataFrame to be saved
        :param path: Destination path
        :param num_partitions: Number of partitions for saving data (default is 1)
        :param file_format: File format to save the data (default is "parquet")
        """
        
        # Repartitioning the DataFrame based on user input
        df = df.repartition(num_partitions)
        
        # Saving the DataFrame to the specified path and format
        df.write.mode('overwrite').format(file_format).save(path)
        
    @staticmethod
    def download_nomes(save_base_path="./output/nomes"):        
        """
        Baixa e extrai o arquivo nomes.csv.gz do dataset genero-nomes no Brasil.io.

        Parâmetros:
            save_base_path (str, opcional): Caminho base onde o arquivo será salvo. O padrão é './output/nomes'.

        Descrição:
            - Cria o diretório de salvamento se ele não existir.
            - Baixa o arquivo nomes.csv.gz da URL especificada.
            - Extrai o conteúdo do arquivo .gz.
            - Remove o arquivo .gz original, mantendo apenas o arquivo CSV extraído.

        Notas:
            - Esta função usa a biblioteca `requests` para baixar o arquivo.
            - A função verifica se a resposta do servidor é 200 (sucesso) antes de baixar o arquivo.
            - O arquivo .gz é extraído usando a biblioteca `gzip`.
        """
        # Certifique-se de que o diretório de salvamento exista
        os.makedirs(save_base_path, exist_ok=True)
        url = "https://data.brasil.io/dataset/genero-nomes/nomes.csv.gz"
        # Derive o nome do arquivo da URL
        file_name = os.path.basename(url)
        file_path = os.path.join(save_base_path, file_name)
        extracted_file_path = os.path.join(save_base_path, file_name[:-3])  # remove .gz

        # Baixe o arquivo
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=128):
                    file.write(chunk)
        else:
            print(f"Failed to download {url}. Status code: {response.status_code}")
            return

        # Extraia o arquivo
        with gzip.open(file_path, 'rb') as f_in:
            with open(extracted_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)

        # Apague o arquivo .gz
        os.remove(file_path)
        
    def process_mei(self, df, save_base_path="./output/nomes", file_name="nomes.csv"):
        """
        Processa um DataFrame referente a MEIs, realiza joins com dados adicionais de naturezas jurídicas,
        qualificações, e um conjunto de dados de nomes para extração e categorização de primeiro nome.

        Parâmetros:
            df (pyspark.sql.DataFrame): DataFrame inicial contendo dados sobre MEIs.
            save_base_path (str, opcional): Caminho onde o arquivo com dados de nomes foi extraído. Padrão é './output/nomes'.
            file_name (str, opcional): Nome do arquivo CSV contendo dados de nomes a ser lido. Padrão é 'nomes.csv'.

        Retorna:
            pyspark.sql.DataFrame: DataFrame processado após todas as transformações e joins.

        Descrição:
            1. Realiza join com DataFrames de 'naturezas' e 'qualificações'.
            2. Extração e manipulação de dados de CPF.
            3. Utiliza um dicionário para mapear e criar a coluna "NM_PORTE".
            4. Criptografa a coluna de CPF.
            5. Realiza filtragens baseado na probabilidade do nome ser um CPF válido.
            6. Lê um conjunto de dados de nomes e realiza o explode na coluna 'alternative_names'.
            7. Extrai o primeiro nome da coluna 'NOME_EMPRESA'.
            8. Realiza o join com o conjunto de dados de nomes para categorizar o primeiro nome.
            9. Retorna um DataFrame contendo informações relevantes após todas as transformações.
        """

        df_nat = self.read_data(schema_name='naturezas')
        df_qual = self.read_data(schema_name='qualificacoes')
        
        df = df.withColumnRenamed("QUALIF_RESPONVAVEL", "COD_QUALIFICACAO")
        
        df = df.join(broadcast(df_nat), "COD_NAT_JURICA", "left").drop(df.COD_NAT_JURICA)
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop(df.COD_QUALIFICACAO)
               
        df = df.withColumn("CPF", regexp_replace("NOME_EMPRESA", "[^0-9]", ""))
        df = df.withColumn("CPF", when(col("CPF") == "", None).otherwise(col("CPF")))
        df = df.withColumn('CPF_LEN', length('CPF'))
        # Defina o dicionário de mapeamento
        mapping = {0: 'NÃO INFORMADO',1: 'MICRO EMPRESA',3: ' EMPRESA DE PEQUENO PORTE',5: 'DEMAIS',8: 'BAIXADA'}
        # Use a função 'when' para criar a nova coluna 'NM_SIT_CADASTRAL'
        df = df.withColumn("NM_PORTE",
                           when(df["PORTE"].isin(list(mapping.keys())), df["PORTE"]).otherwise(None))
        
        # Substitua os valores na nova coluna com base no dicionário de mapeamento
        for key, value in mapping.items():
            df = df.withColumn("NM_PORTE", when(df["PORTE"] == key, value).otherwise(df["NM_PORTE"]))

        df = df.withColumn("PROBABILIDADE_DE_SER_CPF", when(df["CPF_LEN"] == 11, "SIM").otherwise("NAO"))
        
        secret_key = secrets.token_urlsafe(24)
        df = df.withColumn("CPF_CRIPTOGRAFADO", expr(f"base64(aes_encrypt(CPF, '{secret_key}', 'ECB', 'PKCS'))")).drop(df.CPF)
        
        # Caminho completo do arquivo
        file_path = os.path.join(save_base_path, file_name)
        
        # Filtrar df_processed baseado na coluna PROBABILIDADE_DE_SER_CPF
        df_filter = df.filter(col('PROBABILIDADE_DE_SER_CPF') == 'SIM').dropDuplicates(subset=['CPF_CRIPTOGRAFADO', 'NOME_EMPRESA'])

        # Ler o arquivo CSV
        df = self.spark.read.csv(file_path, header=True, inferSchema=True)

        # Explodir a coluna alternative_names para múltiplas linhas
        df_expanded = df.withColumn("alternative_names", explode(split(coalesce(col("alternative_names"), col("first_name")), "\\|")))

        # Selecionar as colunas desejadas
        df_result = df_expanded.select("alternative_names", "group_name", "ratio", "classification").dropDuplicates(subset=['alternative_names'])

        # Extrair o primeiro nome da coluna NOME_EMPRESA
        df_filter = df_filter.withColumn("PRIMEIRO_NOME", split(col("NOME_EMPRESA"), " ")[0])
        
        # Fazer o join entre df_filter e df_result
        joined_df = df_filter.join(df_result, df_filter.PRIMEIRO_NOME == df_result.alternative_names, "left").dropDuplicates()
        
        joined_df = joined_df.select('CNPJ','NOME_EMPRESA','CAP_SOCIAL','NM_PORTE','NAT_JURICA','ENTE_FEDERATIVO','NM_QUALIFICACAO','CPF_CRIPTOGRAFADO','CPF_LEN',
                    'PROBABILIDADE_DE_SER_CPF','PRIMEIRO_NOME',col('group_name').alias('GRUPO_NOME'), 
                    col('ratio').alias('PROBABILIDADE_CLASSIFICACAO'), col('classification').alias('CLASSIFICACAO')).dropDuplicates()
        
        return joined_df
    
    def process_socios(self, df, save_base_path="./output/nomes", file_name="nomes.csv"):
        """
        Processa um DataFrame referente a sócios, realiza joins com dados adicionais de países, qualificações, 
        e um conjunto de dados de nomes para extração e categorização de primeiro nome.

        Parâmetros:
            df (pyspark.sql.DataFrame): DataFrame inicial contendo dados sobre sócios.
            save_base_path (str, opcional): Caminho onde o arquivo com dados de nomes foi extraído. Padrão é './output/nomes'.
            file_name (str, opcional): Nome do arquivo CSV contendo dados de nomes a ser lido. Padrão é 'nomes.csv'.

        Retorna:
            pyspark.sql.DataFrame: DataFrame processado após todas as transformações e joins.

        Descrição:
            1. Realiza join com DataFrames de 'países'.
            2. Usa mapeamentos para criar colunas "NM_FAIXA_ETARIA" e "NM_IDENTIFICADOR_SOCIO".
            3. Renomeia e realiza join com DataFrame de qualificações para obter descrições das qualificações.
            4. Converte coluna de data "DATA_ENTRADA_SOCIEDADE" para o formato desejado.
            5. Lê e processa um conjunto de dados de nomes, explodindo e selecionando colunas relevantes.
            6. Extração do primeiro nome da coluna 'NOME_SOCIO_RAZAO_SOCIAL'.
            7. Realiza o join entre o DataFrame processado e o conjunto de dados de nomes para categorizar o primeiro nome.
            8. Retorna um DataFrame contendo informações relevantes após todas as transformações.
        """
        
        df_pais = self.read_data(schema_name='paises')
        df = df.withColumnRenamed("PAIS", "COD_PAIS")
        df = df.join(broadcast(df_pais), "COD_PAIS", "left").drop(df.COD_PAIS)
        
        # Mapeamento de códigos para faixas etárias.
        mapping = {
            1: '0 a 12 anos',
            2: '13 a 20 anos',
            3: '21 a 30 anos',
            4: '31 a 40 anos',
            5: '41 a 50 anos',
            6: '51 a 60 anos',
            7: '61 a 70 anos',
            8: '71 a 80 anos',
            9: 'maiores de 80 anos',
            0: 'NA'
        }
        
                
        # Mapeamento de códigos para faixas etárias.
        id_socio = {
            1: 'PESSOA JURIDICA',
            2: 'PESSOA FISICA',
            3: 'ESTRANGEIRO'}
        
        df_qual = self.read_data(schema_name='qualificacoes')
        
        
        # Use a função 'when' para criar a nova coluna 'NM_SIT_CADASTRAL'
        df = df.withColumn("NM_FAIXA_ETARIA",
                           when(df["FAIXA_ETARIA"].isin(list(mapping.keys())), df["FAIXA_ETARIA"]).otherwise(None))
        
        # Substitua os valores na nova coluna com base no dicionário de mapeamento
        for key, value in mapping.items():
            df = df.withColumn("NM_FAIXA_ETARIA", when(df["FAIXA_ETARIA"] == key, value).otherwise(df["NM_FAIXA_ETARIA"]))
            

        # Use a função 'when' para criar a nova coluna 'NM_SIT_CADASTRAL'
        df = df.withColumn("NM_IDENTIFICADOR_SOCIO",
                           when(df["IDENTIFICADOR_SOCIO"].isin(list(id_socio.keys())), df["IDENTIFICADOR_SOCIO"]).otherwise(None))
        
        # Substitua os valores na nova coluna com base no dicionário de mapeamento
        for key, value in id_socio.items():
            df = df.withColumn("NM_IDENTIFICADOR_SOCIO", when(df["IDENTIFICADOR_SOCIO"] == key, value).otherwise(df["NM_IDENTIFICADOR_SOCIO"]))


        df_qual = self.read_data(schema_name='qualificacoes')
        # Renomeação e join com df_qual para obter descrições de qualificações.
        df = df.withColumnRenamed("QUALIFICACAO_REPRESENTANTE_LEGAL", "COD_QUALIFICACAO")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO")
        df = df.withColumnRenamed("NM_QUALIFICACAO", "NM_QUALIFICACAO_REPRESENTANTE_LEGAL")

        df = df.withColumnRenamed("QUALIFICAÇAO_SOCIO", "COD_QUALIFICACAO")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO")
        df = df.withColumnRenamed("NM_QUALIFICACAO", "NM_QUALIFICAÇAO_SOCIO")

        # Conversão da coluna de data.
        df = df.withColumn("DT_ENTRADA_SOCIEDADE", to_date(col('DATA_ENTRADA_SOCIEDADE'), "yyyyMMdd")).drop(df.DATA_ENTRADA_SOCIEDADE)

        # Leitura do arquivo CSV.
        file_path = os.path.join(save_base_path, file_name)
        df_csv = self.spark.read.csv(file_path, header=True, inferSchema=True)

        df_csv = df_csv.withColumn("alternative_name2", explode(split(df_csv["alternative_names"], "\|")))
        df_result = df_csv.select("alternative_name2", "group_name", "ratio", "classification").dropDuplicates(["alternative_name2"])

        # Extração do primeiro nome.
        df = df.withColumn("PRIMEIRO_NOME", split(col("NOME_SOCIO_RAZAO_SOCIAL"), " ")[0]).dropDuplicates()

        # Join entre dataframes.
        joined_df = df.join(df_result, df.PRIMEIRO_NOME == df_result.alternative_name2, "left").dropDuplicates()
        
        joined_df = joined_df.select('CNPJ_BASICO','NOME_SOCIO_RAZAO_SOCIAL','CNPJ_CPF_SOCIO','REPRESENTANTE_LEGAL',
        'NOME_REPRESENTANTE','NM_PAIS','NM_FAIXA_ETARIA','NM_IDENTIFICADOR_SOCIO','NM_QUALIFICACAO_REPRESENTANTE_LEGAL',
        'NM_QUALIFICAÇAO_SOCIO','DT_ENTRADA_SOCIEDADE','PRIMEIRO_NOME',col('ratio').alias('PROBABILIDADE_CLASSIFICACAO'),
        col('classification').alias('CLASSIFICACAO')).dropDuplicates()

        return joined_df

In [9]:
class ReceitaCNPJApi:
    """
    Classe ReceitaCNPJApi:

    Esta classe é responsável por interagir com a API de Dados Abertos da Receita Federal, facilitando 
    a obtenção de informações relacionadas a CNPJs, como empresas, sócios, municípios, entre outros. 
    A classe inclui funcionalidades para baixar, descompactar e salvar dados, além de utilitários 
    para manipular e consultar URLs.
    """

    # URL base da API de dados abertos da Receita Federal.
    BASE_URL = "https://dadosabertos.rfb.gov.br/CNPJ"

    # Prefixos de arquivos que podem ser baixados da API.
    FILE_PREFIXES = ['Estabelecimentos', 'Municipios', 'Simples', 'Empresas', 'Cnaes', 'Socios', 'Naturezas','Qualificacoes','Paises','Motivos']

    # Número máximo de tentativas ao fazer uma solicitação para a API.
    MAX_ATTEMPTS = 15

    # Tempo de espera entre tentativas de solicitações (em segundos).
    WAIT_TIME = 180

    def __init__(self):
        """
        Inicializador da classe. Configura o logger para registrar atividades e erros.
        """
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO)

    @staticmethod
    def get_last_month():
        """
        Retorna a data atual.
        """
        return (datetime.today()).date()

    def get_output_path_for_prefix(self, prefix):
        """
        Gera o caminho completo de saída para armazenar os dados com base em um prefixo fornecido.

        Args:
        prefix (str): Prefixo do arquivo para o qual o caminho de saída será gerado.

        Returns:
        str: Caminho completo de saída para o prefixo especificado.
        """
        return os.path.join(self.BASE_OUTPUT_PATH, prefix, f"{prefix}.csv")

    def get_most_common_date(self):
        """
        Consulta a API para obter a data mais comum em que os arquivos foram atualizados.
        """

        # Etapa 1: Tentar consultar a API e obter uma resposta.
        try:
            response = requests.get(self.BASE_URL)
            response.raise_for_status()

            # Etapa 2: Usar a biblioteca BeautifulSoup para analisar a resposta HTML.
            soup = BeautifulSoup(response.text, 'html.parser')

            # Etapa 3: Selecionar todos os elementos de data da resposta HTML.
            # Assumindo que a data é o terceiro elemento filho da tag 'tr'.
            date_elements = soup.select('tr > td:nth-child(3)')

            # Etapa 4: Transformar cada elemento de data em um objeto datetime.
            dates = [datetime.strptime(elem.get_text().strip(), '%Y-%m-%d %H:%M') for elem in date_elements if elem.get_text().strip() != '']

            # Etapa 5: Contar as ocorrências de cada data e determinar a data mais comum.
            most_common_date, _ = Counter([date.date() for date in dates]).most_common(1)[0]

            # Etapa 6: Retornar a data mais comum.
            return datetime.combine(most_common_date, datetime.min.time()).date()
        except requests.RequestException as e:
            # Etapa 7: Em caso de qualquer erro na consulta da API, registrar o erro e retornar None.
            self.logger.error(f"Failed to get most common date. Error: {e}")
            return None

    def lista_urls_receita(self, *prefixes):
        """
        Gera uma lista de URLs para download com base nos prefixos fornecidos. As URLs são 
        determinadas com base nos prefixos de arquivo conhecidos. Se nenhum prefixo for 
        fornecido, o método gerará URLs para todos os tipos de arquivos conhecidos.

        Args:
        *prefixes (str): Prefixos de arquivos para os quais as URLs serão geradas.

        Returns:
        list: Lista de URLs completas para os arquivos correspondentes aos prefixos.
        """
        # Etapa 1: Inicializar a lista vazia para armazenar as URLs.
        urls = []

        # Etapa 2: Se nenhum prefixo for fornecido como argumento, use todos os prefixos de arquivo conhecidos.
        if not prefixes:
            prefixes = self.FILE_PREFIXES

        # Etapa 3: Iterar sobre cada prefixo fornecido.
        for prefix in prefixes:
            # Etapa 3.1: Se o prefixo estiver na lista de prefixos especificados, 
            # adicione a URL correspondente à lista de URLs.
            if prefix in ['Municipios', 'Cnaes', 'Naturezas', 'Simples','Qualificacoes','Paises','Motivos']:
                urls.append(f"{self.BASE_URL}/{prefix}.zip")

            # Etapa 3.2: Se o prefixo estiver na lista de prefixos de arquivo conhecidos, 
            # gere URLs para cada arquivo (de 0 a 9) e adicione-as à lista de URLs.
            elif prefix in self.FILE_PREFIXES:
                urls.extend([f"{self.BASE_URL}/{prefix}{i}.zip" for i in range(10)])

            # Etapa 3.3: Se o prefixo fornecido não for reconhecido, 
            # imprima uma mensagem informando que o prefixo não é reconhecido.
            else:
                print(f"Prefixo '{prefix}' não reconhecido!")

        # Etapa 4: Retorne a lista completa de URLs geradas.
        return urls

    def fetch_data(self, url, save_path, log_accumulator=None, max_attempts=15, wait_time=180):
        """
        Baixa um arquivo da URL especificada e o salva no caminho de saída especificado. Em caso de 
        falha na tentativa de download, tentará novamente até o número máximo de tentativas ser atingido.

        Args:
        url (str): URL de onde o arquivo será baixado.
        save_path (str): Caminho onde o arquivo baixado será salvo.
        log_accumulator (list, optional): Um acumulador para armazenar mensagens de log. Padrão para None.
        max_attempts (int, optional): Número máximo de tentativas de download. Padrão para 15.
        wait_time (int, optional): Tempo de espera entre tentativas em segundos. Padrão para 150.

        Raises:
        Exception: Se o número máximo de tentativas for atingido sem sucesso.

        Returns:
        str: Caminho completo do arquivo baixado.
        """

        # Etapa 1: Criar o diretório no caminho de salvamento, caso ele não exista.
        os.makedirs(save_path, exist_ok=True)

        # Etapa 2: Derivar o nome do arquivo da URL fornecida.
        file_name = url.split('/')[-1]
        file_path = os.path.join(save_path, file_name)

        # Etapa 3: Definir os cabeçalhos da solicitação.
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

        # Etapa 4: Iniciar tentativas de download do arquivo.
        for attempt in range(max_attempts):
            try:
                # Etapa 4.1: Fazer uma solicitação GET para a URL.
                response = requests.get(url, headers=headers)
                response.raise_for_status()

                # Etapa 4.2: Escrever o conteúdo da resposta no arquivo.
                with open(file_path, 'wb') as file:
                    file.write(response.content)
                return file_path

            except requests.RequestException as e:
                # Etapa 4.3: Registrar falha na tentativa de download.
                msg = f"Tentativa {attempt + 1} de {max_attempts} falhou. Erro: {e}"
                if log_accumulator:
                    log_accumulator.append([msg, "ERRO NO REQUEST! RETRY SENDO FEITO"])

                # Etapa 4.4: Se for a última tentativa, registrar o erro persistente e lançar uma exceção.
                if attempt == max_attempts - 1:
                    if log_accumulator:
                        log_accumulator.append(["ERRO PERSISTENTE AO TENTAR BAIXAR O ARQUIVO"])
                    raise

                # Etapa 4.5: Aguardar o tempo especificado antes de tentar novamente.
                time.sleep(wait_time)

                
    def download_and_unzip(self, url, save_base_path="./temp", output_base_path="./output", headers=None, log_accumulator=None, data_update=True):
        """
        Baixa um arquivo zip da URL fornecida, descompacta e salva no caminho especificado. Se os 
        dados da Receita Federal não foram atualizados nos últimos 30 dias e o parâmetro data_update 
        estiver ativado, uma mensagem de log será gerada.

        Args:
        url (str): URL de onde o arquivo zip será baixado.
        save_base_path (str, optional): Caminho base onde o arquivo baixado será salvo. Padrão para "./temp".
        output_base_path (str, optional): Caminho base onde os dados descompactados serão armazenados. Padrão para "./output".
        headers (dict, optional): Cabeçalhos HTTP para serem usados no pedido. Padrão para None.
        log_accumulator (list, optional): Uma acumulador para armazenar mensagens de log. Padrão para None.
        data_update (bool, optional): Determina se a função deve verificar se os dados foram atualizados nos últimos 30 dias. Padrão para True.

        Returns:
        tuple: Um tuple contendo a URL e uma mensagem indicando "Success" ou a razão da falha.
        """

        # Etapa 1: Identificar o nome do arquivo e seu prefixo a partir da URL fornecida.
        file_name = url.split('/')[-1]
        prefix = next((p for p in self.FILE_PREFIXES if file_name.startswith(p)), None)
        if not prefix:
            return (url, "Failed to determine prefix")

        # Etapa 2: Configurar o caminho de salvamento e garantir que o diretório de salvamento exista.
        save_path = os.path.join(save_base_path)
        os.makedirs(save_path, exist_ok=True)

        # Etapa 3: Obter a data mais comum de atualização dos arquivos da API e verificar se é válida.
        data_atualizacao = self.get_most_common_date()
        if data_atualizacao is None:
            return (url, "Failed to determine most common date")

        # Etapa 4: Obter a data atual.
        data_atual = self.get_last_month()

        # Etapa 5: Verificar se os dados da Receita Federal foram atualizados nos últimos 30 dias.
        if not data_update or (data_update and (data_atual - data_atualizacao).days <= 30):
            try:
                # Etapa 5.1: Tentar baixar o arquivo zip da URL fornecida.
                zip_file_path = self.fetch_data(url, save_path, headers)

                # Etapa 5.2: Configurar o caminho de saída e garantir que o diretório de saída exista.
                output_path = os.path.join(output_base_path)
                os.makedirs(output_path, exist_ok=True)

                # Etapa 5.3: Tentar descompactar o arquivo baixado.
                self.unzip_files(zip_file_path, output_path)

                # Etapa 5.4: Registrar a conclusão bem-sucedida da descompactação.
                self.logger.info(f"File {url} has been unzipped successfully to {output_path}")
                return (url, "Success")
            except Exception as e:
                # Etapa 5.5: Em caso de qualquer erro, registrar o erro e retornar a mensagem.
                self.logger.error(f"Failed to download and unzip {url}. Error: {e}")
                return (url, str(e))
        else:
            # Etapa 6: Se os dados não foram atualizados nos últimos 30 dias e data_update está ativado, registrar uma mensagem.
            log_msg = "Os dados da receita federal não foram atualizados nos últimos 30 dias"
            if log_accumulator:
                log_accumulator.add([log_msg])
            self.logger.info(log_msg)
            return (url, log_msg)

    def unzip_files(self, zip_file_path, output_base_path, log_accumulator=None):
        """
        Descompacta o arquivo fornecido e salva no caminho especificado. Substitui arquivos existentes.

        Args:
        - zip_file_path (str): Caminho completo do arquivo zip que precisa ser descompactado.
        - output_base_path (str): Caminho base onde o arquivo descompactado deve ser salvo.
        - log_accumulator (list, optional): Uma lista que pode ser fornecida para acumular mensagens de log, útil para rastrear erros ou informações. Se não for fornecido, apenas os logs padrão serão usados.

        """

        # Etapa 1: Extraia o nome do arquivo zip da rota fornecida.
        zip_file_name = os.path.basename(zip_file_path)

        # Etapa 2: Determine o prefixo do arquivo com base nos prefixos conhecidos.
        prefix = next((p for p in self.FILE_PREFIXES if zip_file_name.startswith(p)), None)
        if not prefix:
            msg = f"File {zip_file_path} does not match expected patterns."

            # Etapa 2.1: Se o prefixo não for encontrado, adicione uma mensagem de erro ao acumulador de log (se fornecido) e retorne.
            if log_accumulator:
                log_accumulator.append(msg)
            self.logger.error(msg)
            return

        # Etapa 3: Configurar o caminho de saída e garantir que o diretório de saída exista.
        output_path = os.path.join(output_base_path, prefix)
        os.makedirs(output_path, exist_ok=True)

        # Etapa 4: Abrir o arquivo zip.
        self.logger.info(f"Trying to unzip {zip_file_path}")
        with zipfile.ZipFile(zip_file_path, "r") as z:

            # Etapa 4.1: Pegue o nome do primeiro arquivo dentro do arquivo zip.
            file_inside_zip = z.namelist()[0]

            # Etapa 4.2: Extraia a parte numérica do nome do arquivo zip para nomear corretamente o arquivo csv resultante.
            number_in_zip = ''.join(filter(str.isdigit, zip_file_name))
            final_file_path = os.path.join(output_path, f"{prefix}{number_in_zip}.csv")

            # Etapa 4.3: Se o arquivo csv já existir no caminho de destino, exclua-o para garantir que o novo arquivo não seja sobreposto.
            if os.path.exists(final_file_path):
                os.remove(final_file_path)

            # Etapa 4.4: Descompacte o conteúdo do arquivo zip diretamente para o caminho de saída desejado.
            with z.open(file_inside_zip) as zf, open(final_file_path, 'wb') as f_out:
                shutil.copyfileobj(zf, f_out)

        # Etapa 5: Exclua o arquivo zip original após a extração.
        os.remove(zip_file_path)


In [1]:
import os
import logging
import requests
import zipfile
import shutil
import time
from datetime import datetime
from collections import Counter
from bs4 import BeautifulSoup

class ReceitaCNPJApi:
    """
    Classe ReceitaCNPJApi:

    Esta classe é responsável por interagir com a API de Dados Abertos da Receita Federal, facilitando 
    a obtenção de informações relacionadas a CNPJs, como empresas, sócios, municípios, entre outros. 
    A classe inclui funcionalidades para baixar, descompactar e salvar dados, além de utilitários 
    para manipular e consultar URLs.
    """

    # Prefixos de arquivos que podem ser baixados da API.
    FILE_PREFIXES = [
        'Estabelecimentos', 'Municipios', 'Simples', 'Empresas', 
        'Cnaes', 'Socios', 'Naturezas', 'Qualificacoes', 'Paises', 'Motivos'
    ]

    # Número máximo de tentativas ao fazer uma solicitação para a API.
    MAX_ATTEMPTS = 15

    # Tempo de espera entre tentativas de solicitações (em segundos).
    WAIT_TIME = 180

    # URL base da API de dados abertos da Receita Federal, sem incluir o ano/mês.
    BASE_URL = "https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj"

    def __init__(self, base_output_path: str = "./output", save_base_path: str = "./temp"):
        """
        Inicializador da classe. Configura o logger para registrar atividades e erros e define os caminhos base 
        para salvar os arquivos descompactados e baixados.

        Args:
            base_output_path (str, optional): Caminho base para salvar os arquivos descompactados. Padrão para "./output".
            save_base_path (str, optional): Caminho base para salvar os arquivos baixados. Padrão para "./temp".
        """
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO)

        self.BASE_OUTPUT_PATH = base_output_path
        self.SAVE_BASE_PATH = save_base_path

    @staticmethod
    def get_current_date():
        """
        Retorna a data atual.

        Returns:
            date: Data atual.
        """
        return datetime.today().date()

    def get_output_path_for_prefix(self, prefix):
        """
        Gera o caminho completo de saída para armazenar os dados com base em um prefixo fornecido.

        Args:
            prefix (str): Prefixo do arquivo para o qual o caminho de saída será gerado.

        Returns:
            str: Caminho completo de saída para o prefixo especificado.
        """
        return os.path.join(self.BASE_OUTPUT_PATH, prefix, f"{prefix}.csv")

    def lista_urls_receita(self, year: int = None, month: int = None, *prefixes):
        """
        Gera uma lista de URLs para download com base nos prefixos fornecidos e no período especificado. 
        Se nenhum prefixo for fornecido, o método gerará URLs para todos os tipos de arquivos conhecidos.

        Args:
            year (int, optional): Ano de atualização dos dados (ex: 2024). Se não fornecido, usa o ano atual.
            month (int, optional): Mês de atualização dos dados (ex: 9 para setembro). Se não fornecido, usa o mês atual.
            *prefixes (str): Prefixos de arquivos para os quais as URLs serão geradas.

        Returns:
            list: Lista de URLs completas para os arquivos correspondentes aos prefixos.
        """
        # Determinar o ano e o mês a serem usados
        if year is None or month is None:
            current_date = self.get_current_date()
            year = year or current_date.year
            month = month or current_date.month

        # Validar os parâmetros de ano e mês
        if not (1 <= month <= 12):
            raise ValueError("O mês deve estar entre 1 e 12.")
        if year < 0:
            raise ValueError("O ano deve ser um valor positivo.")

        # Construir a URL completa com base no ano e mês
        period = f"{year:04d}-{month:02d}"
        full_base_url = f"{self.BASE_URL}/{period}"

        self.logger.info(f"Gerando URLs para o período: {period}")

        urls = []

        # Se nenhum prefixo for fornecido, usar todos os prefixos conhecidos
        if not prefixes:
            prefixes = self.FILE_PREFIXES

        for prefix in prefixes:
            if prefix in ['Municipios', 'Cnaes', 'Naturezas', 'Simples', 'Qualificacoes', 'Paises', 'Motivos']:
                urls.append(f"{full_base_url}/{prefix}.zip")
            elif prefix in self.FILE_PREFIXES:
                urls.extend([f"{full_base_url}/{prefix}{i}.zip" for i in range(10)])
            else:
                self.logger.warning(f"Prefixo '{prefix}' não reconhecido!")

        self.logger.info(f"{len(urls)} URLs geradas.")
        return urls

    def fetch_data(self, url, log_accumulator=None, max_attempts=15, wait_time=180):
        """
        Baixa um arquivo da URL especificada e o salva no caminho de saída especificado. Em caso de 
        falha na tentativa de download, tentará novamente até o número máximo de tentativas ser atingido.

        Args:
            url (str): URL de onde o arquivo será baixado.
            log_accumulator (list, optional): Um acumulador para armazenar mensagens de log. Padrão para None.
            max_attempts (int, optional): Número máximo de tentativas de download. Padrão para 15.
            wait_time (int, optional): Tempo de espera entre tentativas em segundos. Padrão para 180.

        Raises:
            Exception: Se o número máximo de tentativas for atingido sem sucesso.

        Returns:
            str: Caminho completo do arquivo baixado.
        """
        os.makedirs(self.SAVE_BASE_PATH, exist_ok=True)

        file_name = url.split('/')[-1]
        file_path = os.path.join(self.SAVE_BASE_PATH, file_name)

        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.124 Safari/537.36"
            )
        }

        for attempt in range(1, max_attempts + 1):
            try:
                self.logger.info(f"Baixando {url} (Tentativa {attempt}/{max_attempts})")
                response = requests.get(url, headers=headers)
                response.raise_for_status()

                with open(file_path, 'wb') as file:
                    file.write(response.content)
                self.logger.info(f"Download concluído: {file_path}")
                return file_path

            except requests.RequestException as e:
                msg = f"Tentativa {attempt} de {max_attempts} falhou. Erro: {e}"
                self.logger.error(msg)
                if log_accumulator is not None:
                    log_accumulator.append([msg, "ERRO NO REQUEST! RETRY SENDO FEITO"])

                if attempt == max_attempts:
                    final_msg = "Número máximo de tentativas atingido. Download falhou."
                    self.logger.error(final_msg)
                    if log_accumulator is not None:
                        log_accumulator.append([final_msg])
                    raise Exception(final_msg) from e

                self.logger.info(f"Aguardando {wait_time} segundos antes da próxima tentativa...")
                time.sleep(wait_time)

    def download_and_unzip(self, url, output_base_path=None, log_accumulator=None):
        """
        Baixa um arquivo zip da URL fornecida, descompacta e salva no caminho especificado.

        Args:
            url (str): URL de onde o arquivo zip será baixado.
            output_base_path (str, optional): Caminho base onde os dados descompactados serão armazenados. 
                                              Se não for fornecido, usa o caminho definido no inicializador.
            log_accumulator (list, optional): Um acumulador para armazenar mensagens de log. Padrão para None.

        Returns:
            tuple: Um tuple contendo a URL e uma mensagem indicando "Success" ou a razão da falha.
        """
        if output_base_path is None:
            output_base_path = self.BASE_OUTPUT_PATH

        try:
            zip_file_path = self.fetch_data(url, log_accumulator)
            self.unzip_files(zip_file_path, output_base_path, log_accumulator)
            self.logger.info(f"Arquivo {url} foi descompactado com sucesso para {output_base_path}")
            return (url, "Success")
        except Exception as e:
            error_msg = f"Falha ao baixar e descompactar {url}. Erro: {e}"
            self.logger.error(error_msg)
            return (url, str(e))

    def unzip_files(self, zip_file_path, output_base_path, log_accumulator=None):
        """
        Descompacta o arquivo fornecido e salva no caminho especificado. Substitui arquivos existentes.

        Args:
            zip_file_path (str): Caminho completo do arquivo zip que precisa ser descompactado.
            output_base_path (str): Caminho base onde o arquivo descompactado deve ser salvo.
            log_accumulator (list, optional): Uma lista que pode ser fornecida para acumular mensagens de log, 
                                             útil para rastrear erros ou informações. Se não for fornecido, 
                                             apenas os logs padrão serão usados.
        """
        zip_file_name = os.path.basename(zip_file_path)
        prefix = next((p for p in self.FILE_PREFIXES if zip_file_name.startswith(p)), None)

        if not prefix:
            msg = f"O arquivo {zip_file_path} não corresponde aos padrões esperados."
            self.logger.error(msg)
            if log_accumulator is not None:
                log_accumulator.append([msg])
            return

        output_path = os.path.join(output_base_path, prefix)
        os.makedirs(output_path, exist_ok=True)

        try:
            self.logger.info(f"Descompactando {zip_file_path} para {output_path}")
            with zipfile.ZipFile(zip_file_path, "r") as z:
                file_inside_zip = z.namelist()[0]
                number_in_zip = ''.join(filter(str.isdigit, zip_file_name))
                final_file_path = os.path.join(output_path, f"{prefix}{number_in_zip}.csv")

                if os.path.exists(final_file_path):
                    os.remove(final_file_path)
                    self.logger.info(f"Arquivo existente {final_file_path} removido.")

                with z.open(file_inside_zip) as zf, open(final_file_path, 'wb') as f_out:
                    shutil.copyfileobj(zf, f_out)
            self.logger.info(f"Arquivo descompactado com sucesso: {final_file_path}")
        except zipfile.BadZipFile as e:
            msg = f"Erro ao descompactar {zip_file_path}: {e}"
            self.logger.error(msg)
            if log_accumulator is not None:
                log_accumulator.append([msg])
            raise
        finally:
            if os.path.exists(zip_file_path):
                os.remove(zip_file_path)
                self.logger.info(f"Arquivo zip {zip_file_path} removido após descompactação.")

In [9]:
import os
import logging
import requests
import zipfile
import shutil
import time
from datetime import datetime
from collections import Counter
from bs4 import BeautifulSoup

class ReceitaCNPJApi:
    """
    Classe ReceitaCNPJApi:

    Esta classe é responsável por interagir com a API de Dados Abertos da Receita Federal, facilitando 
    a obtenção de informações relacionadas a CNPJs, como empresas, sócios, municípios, entre outros. 
    A classe inclui funcionalidades para baixar, descompactar e salvar dados, além de utilitários 
    para manipular e consultar URLs.
    """

    # Prefixos de arquivos que podem ser baixados da API.
    FILE_PREFIXES = [
        'Estabelecimentos', 'Municipios', 'Simples', 'Empresas', 
        'Cnaes', 'Socios', 'Naturezas', 'Qualificacoes', 'Paises', 'Motivos'
    ]

    # Número máximo de tentativas ao fazer uma solicitação para a API.
    MAX_ATTEMPTS = 15

    # Tempo de espera entre tentativas de solicitações (em segundos).
    WAIT_TIME = 180

    # URL base da API de dados abertos da Receita Federal, sem incluir o ano/mês.
    BASE_URL = "https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj"

    def __init__(self, base_output_path: str = "./output", save_base_path: str = "./temp"):
        """
        Inicializador da classe. Configura o logger para registrar atividades e erros e define os caminhos base 
        para salvar os arquivos descompactados e baixados.

        Args:
            base_output_path (str, optional): Caminho base para salvar os arquivos descompactados. Padrão para "./output".
            save_base_path (str, optional): Caminho base para salvar os arquivos baixados. Padrão para "./temp".
        """
        self.logger = logging.getLogger(__name__)
        logging.basicConfig(level=logging.INFO)

        self.BASE_OUTPUT_PATH = base_output_path
        self.SAVE_BASE_PATH = save_base_path

    @staticmethod
    def get_current_date():
        """
        Retorna a data atual.

        Returns:
            date: Data atual.
        """
        return datetime.today().date()

    def get_output_path_for_prefix(self, prefix):
        """
        Gera o caminho completo de saída para armazenar os dados com base em um prefixo fornecido.

        Args:
            prefix (str): Prefixo do arquivo para o qual o caminho de saída será gerado.

        Returns:
            str: Caminho completo de saída para o prefixo especificado.
        """
        return os.path.join(self.BASE_OUTPUT_PATH, prefix, f"{prefix}.csv")

    def lista_urls_receita(self, *prefixes, year: int = None, month: int = None):
        """
        Gera uma lista de URLs para download com base nos prefixos fornecidos e no período especificado. 
        Se nenhum prefixo for fornecido, o método gerará URLs para todos os tipos de arquivos conhecidos.

        Args:
            *prefixes (str): Prefixos de arquivos para os quais as URLs serão geradas.
            year (int, optional): Ano de atualização dos dados (ex: 2024). Se não fornecido, usa o ano atual.
            month (int, optional): Mês de atualização dos dados (ex: 9 para setembro). Se não fornecido, usa o mês atual.

        Returns:
            list: Lista de URLs completas para os arquivos correspondentes aos prefixos.
        """
        # Determinar o ano e o mês a serem usados
        if year is None or month is None:
            current_date = self.get_current_date()
            year = year or current_date.year
            month = month or current_date.month

        # Validar os parâmetros de ano e mês
        if not (1 <= month <= 12):
            raise ValueError("O mês deve estar entre 1 e 12.")
        if year < 0:
            raise ValueError("O ano deve ser um valor positivo.")

        # Construir a URL completa com base no ano e mês
        period = f"{year:04d}-{month:02d}"
        full_base_url = f"{self.BASE_URL}/{period}"

        self.logger.info(f"Gerando URLs para o período: {period}")

        urls = []

        # Se nenhum prefixo for fornecido, usar todos os prefixos conhecidos
        if not prefixes:
            prefixes = self.FILE_PREFIXES

        for prefix in prefixes:
            if prefix in ['Municipios', 'Cnaes', 'Naturezas', 'Simples', 'Qualificacoes', 'Paises', 'Motivos']:
                urls.append(f"{full_base_url}/{prefix}.zip")
            elif prefix in self.FILE_PREFIXES:
                urls.extend([f"{full_base_url}/{prefix}{i}.zip" for i in range(10)])
            else:
                self.logger.warning(f"Prefixo '{prefix}' não reconhecido!")

        self.logger.info(f"{len(urls)} URLs geradas.")
        return urls

    def fetch_data(self, url, log_accumulator=None, max_attempts=15, wait_time=180):
        """
        Baixa um arquivo da URL especificada e o salva no caminho de saída especificado. Em caso de 
        falha na tentativa de download, tentará novamente até o número máximo de tentativas ser atingido.

        Args:
            url (str): URL de onde o arquivo será baixado.
            log_accumulator (list, optional): Um acumulador para armazenar mensagens de log. Padrão para None.
            max_attempts (int, optional): Número máximo de tentativas de download. Padrão para 15.
            wait_time (int, optional): Tempo de espera entre tentativas em segundos. Padrão para 180.

        Raises:
            Exception: Se o número máximo de tentativas for atingido sem sucesso.

        Returns:
            str: Caminho completo do arquivo baixado.
        """
        os.makedirs(self.SAVE_BASE_PATH, exist_ok=True)

        file_name = url.split('/')[-1]
        file_path = os.path.join(self.SAVE_BASE_PATH, file_name)

        headers = {
            "User-Agent": (
                "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                "AppleWebKit/537.36 (KHTML, like Gecko) "
                "Chrome/91.0.4472.124 Safari/537.36"
            )
        }

        for attempt in range(1, max_attempts + 1):
            try:
                self.logger.info(f"Baixando {url} (Tentativa {attempt}/{max_attempts})")
                response = requests.get(url, headers=headers)
                response.raise_for_status()

                with open(file_path, 'wb') as file:
                    file.write(response.content)
                self.logger.info(f"Download concluído: {file_path}")
                return file_path

            except requests.RequestException as e:
                msg = f"Tentativa {attempt} de {max_attempts} falhou. Erro: {e}"
                self.logger.error(msg)
                if log_accumulator is not None:
                    log_accumulator.append([msg, "ERRO NO REQUEST! RETRY SENDO FEITO"])

                if attempt == max_attempts:
                    final_msg = "Número máximo de tentativas atingido. Download falhou."
                    self.logger.error(final_msg)
                    if log_accumulator is not None:
                        log_accumulator.append([final_msg])
                    raise Exception(final_msg) from e

                self.logger.info(f"Aguardando {wait_time} segundos antes da próxima tentativa...")
                time.sleep(wait_time)

    def download_and_unzip(self, url, output_base_path=None, log_accumulator=None):
        """
        Baixa um arquivo zip da URL fornecida, descompacta e salva no caminho especificado.

        Args:
            url (str): URL de onde o arquivo zip será baixado.
            output_base_path (str, optional): Caminho base onde os dados descompactados serão armazenados. 
                                              Se não for fornecido, usa o caminho definido no inicializador.
            log_accumulator (list, optional): Um acumulador para armazenar mensagens de log. Padrão para None.

        Returns:
            tuple: Um tuple contendo a URL e uma mensagem indicando "Success" ou a razão da falha.
        """
        if output_base_path is None:
            output_base_path = self.BASE_OUTPUT_PATH

        try:
            zip_file_path = self.fetch_data(url, log_accumulator)
            self.unzip_files(zip_file_path, output_base_path, log_accumulator)
            self.logger.info(f"Arquivo {url} foi descompactado com sucesso para {output_base_path}")
            return (url, "Success")
        except Exception as e:
            error_msg = f"Falha ao baixar e descompactar {url}. Erro: {e}"
            self.logger.error(error_msg)
            return (url, str(e))

    def unzip_files(self, zip_file_path, output_base_path, log_accumulator=None):
        """
        Descompacta o arquivo fornecido e salva no caminho especificado. Substitui arquivos existentes.

        Args:
            zip_file_path (str): Caminho completo do arquivo zip que precisa ser descompactado.
            output_base_path (str): Caminho base onde o arquivo descompactado deve ser salvo.
            log_accumulator (list, optional): Uma lista que pode ser fornecida para acumular mensagens de log, 
                                             útil para rastrear erros ou informações. Se não for fornecido, 
                                             apenas os logs padrão serão usados.
        """
        zip_file_name = os.path.basename(zip_file_path)
        prefix = next((p for p in self.FILE_PREFIXES if zip_file_name.startswith(p)), None)

        if not prefix:
            msg = f"O arquivo {zip_file_path} não corresponde aos padrões esperados."
            self.logger.error(msg)
            if log_accumulator is not None:
                log_accumulator.append([msg])
            return

        output_path = os.path.join(output_base_path, prefix)
        os.makedirs(output_path, exist_ok=True)

        try:
            self.logger.info(f"Descompactando {zip_file_path} para {output_path}")
            with zipfile.ZipFile(zip_file_path, "r") as z:
                file_inside_zip = z.namelist()[0]
                number_in_zip = ''.join(filter(str.isdigit, zip_file_name))
                final_file_path = os.path.join(output_path, f"{prefix}{number_in_zip}.csv")

                if os.path.exists(final_file_path):
                    os.remove(final_file_path)
                    self.logger.info(f"Arquivo existente {final_file_path} removido.")

                with z.open(file_inside_zip) as zf, open(final_file_path, 'wb') as f_out:
                    shutil.copyfileobj(zf, f_out)
            self.logger.info(f"Arquivo descompactado com sucesso: {final_file_path}")
        except zipfile.BadZipFile as e:
            msg = f"Erro ao descompactar {zip_file_path}: {e}"
            self.logger.error(msg)
            if log_accumulator is not None:
                log_accumulator.append([msg])
            raise
        finally:
            if os.path.exists(zip_file_path):
                os.remove(zip_file_path)
                self.logger.info(f"Arquivo zip {zip_file_path} removido após descompactação.")


## Extrando de forma uni thread

In [10]:
# Inicializa a classe sem parâmetros
api = ReceitaCNPJApi()

# Especifica o ano e o mês desejados
year = 2024
month = 9

# Gera a lista de URLs para todos os prefixos no período especificado
urls = api.lista_urls_receita(year=year, month=month)

for url in urls:
    print(f"Result for {url}")

INFO:__main__:Gerando URLs para o período: 2024-09
INFO:__main__:37 URLs geradas.


Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos0.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos1.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos2.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos3.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos4.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos5.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos6.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos7.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos8.zip
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos9.zip
Result for https://dadosaberto

In [ ]:
# Inicializa a classe sem parâmetros
api = ReceitaCNPJApi()

# Especifica o ano e o mês desejados
year = 2024
month = 9

# Gera a lista de URLs para todos os prefixos no período especificado
urls = api.lista_urls_receita(year=year, month=month)

for url in urls:
    result = api.download_and_unzip(url)
    print(f"Result for {url}: {result[1]}")

INFO:__main__:Gerando URLs para o período: 2024-09
INFO:__main__:37 URLs geradas.
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos0.zip (Tentativa 1/15)


## Usando multi thread do python

In [11]:
from concurrent.futures import ThreadPoolExecutor

# Inicializa a classe sem parâmetros
api = ReceitaCNPJApi()

# Especifica o ano e o mês desejados
year = 2024
month = 9

# Gera a lista de URLs para múltiplos prefixos no período especificado
urls = api.lista_urls_receita(
    'Municipios', 'Cnaes', 'Naturezas', 'Qualificacoes', 'Paises', 'Motivos',
    year=year,
    month=month
)

# Função para processar cada URL
def process_url(url):
    return api.download_and_unzip(url)

# Baixa e descompacta as URLs utilizando ThreadPoolExecutor para paralelizar
with ThreadPoolExecutor(max_workers=3) as executor:
    results = list(executor.map(process_url, urls))

# Imprime os resultados
for url, result in zip(urls, results):
    print(f"Result for {url}: {result[1]}")


INFO:__main__:Gerando URLs para o período: 2024-09
INFO:__main__:6 URLs geradas.
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Municipios.zip (Tentativa 1/15)
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Cnaes.zip (Tentativa 1/15)
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Naturezas.zip (Tentativa 1/15)
INFO:__main__:Download concluído: ./temp\Naturezas.zip
INFO:__main__:Descompactando ./temp\Naturezas.zip para ./output\Naturezas
INFO:__main__:Arquivo existente ./output\Naturezas\Naturezas.csv removido.
INFO:__main__:Arquivo descompactado com sucesso: ./output\Naturezas\Naturezas.csv
INFO:__main__:Arquivo zip ./temp\Naturezas.zip removido após descompactação.
INFO:__main__:Arquivo https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Naturezas.zip foi descompactado com sucesso para ./output
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/d

Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Municipios.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Cnaes.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Naturezas.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Qualificacoes.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Paises.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Motivos.zip: Success


In [12]:
from concurrent.futures import ThreadPoolExecutor

# Inicializa a classe sem parâmetros
api = ReceitaCNPJApi()

# Especifica o ano e o mês desejados
year = 2024
month = 9

# Gera a lista de URLs apenas para o prefixo 'Empresas' no período especificado
urls = api.lista_urls_receita(
    'Empresas',
    year=year,
    month=month
)

# Função para processar cada URL
def process_url(url):
    return api.download_and_unzip(url)

# Baixa e descompacta as URLs utilizando ThreadPoolExecutor para paralelizar
with ThreadPoolExecutor(max_workers=3) as executor:
    results = list(executor.map(process_url, urls))

# Imprime os resultados
for url, result in zip(urls, results):
    print(f"Result for {url}: {result[1]}")


INFO:__main__:Gerando URLs para o período: 2024-09
INFO:__main__:10 URLs geradas.
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas0.zip (Tentativa 1/15)
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas2.zip (Tentativa 1/15)
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas1.zip (Tentativa 1/15)
INFO:__main__:Download concluído: ./temp\Empresas1.zip
INFO:__main__:Descompactando ./temp\Empresas1.zip para ./output\Empresas
INFO:__main__:Arquivo existente ./output\Empresas\Empresas1.csv removido.
INFO:__main__:Arquivo descompactado com sucesso: ./output\Empresas\Empresas1.csv
INFO:__main__:Arquivo zip ./temp\Empresas1.zip removido após descompactação.
INFO:__main__:Arquivo https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas1.zip foi descompactado com sucesso para ./output
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/

Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas0.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas1.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas2.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas3.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas4.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas5.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas6.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas7.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas8.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Empresas9.zip: Success


In [13]:
from concurrent.futures import ThreadPoolExecutor

# Inicializa a classe sem parâmetros
api = ReceitaCNPJApi()

# Especifica o ano e o mês desejados
year = 2024
month = 9

# Gera a lista de URLs apenas para o prefixo 'Empresas' no período especificado
urls = api.lista_urls_receita(
    'Estabelecimentos',
    year=year,
    month=month
)

# Função para processar cada URL
def process_url(url):
    return api.download_and_unzip(url)

# Baixa e descompacta as URLs utilizando ThreadPoolExecutor para paralelizar
with ThreadPoolExecutor(max_workers=3) as executor:
    results = list(executor.map(process_url, urls))

# Imprime os resultados
for url, result in zip(urls, results):
    print(f"Result for {url}: {result[1]}")


INFO:__main__:Gerando URLs para o período: 2024-09
INFO:__main__:10 URLs geradas.
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos0.zip (Tentativa 1/15)
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos2.zip (Tentativa 1/15)
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos1.zip (Tentativa 1/15)
INFO:__main__:Download concluído: ./temp\Estabelecimentos2.zip
INFO:__main__:Descompactando ./temp\Estabelecimentos2.zip para ./output\Estabelecimentos
INFO:__main__:Arquivo existente ./output\Estabelecimentos\Estabelecimentos2.csv removido.
INFO:__main__:Download concluído: ./temp\Estabelecimentos1.zip
INFO:__main__:Descompactando ./temp\Estabelecimentos1.zip para ./output\Estabelecimentos
INFO:__main__:Arquivo existente ./output\Estabelecimentos\Estabelecimentos1.csv removido.
INFO:__main__:Arquivo descompactado com sucesso: ./outpu

Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos0.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos1.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos2.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos3.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos4.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos5.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos6.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos7.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Estabelecimentos8.zip: Success
Result for https://dadosabertos.rfb.gov.br/CNP

In [14]:
from concurrent.futures import ThreadPoolExecutor

# Inicializa a classe sem parâmetros
api = ReceitaCNPJApi()

# Especifica o ano e o mês desejados
year = 2024
month = 9

# Gera a lista de URLs apenas para o prefixo 'Empresas' no período especificado
urls = api.lista_urls_receita(
    'Simples',
    year=year,
    month=month
)

# Função para processar cada URL
def process_url(url):
    return api.download_and_unzip(url)

# Baixa e descompacta as URLs utilizando ThreadPoolExecutor para paralelizar
with ThreadPoolExecutor(max_workers=1) as executor:
    results = list(executor.map(process_url, urls))

# Imprime os resultados
for url, result in zip(urls, results):
    print(f"Result for {url}: {result[1]}")

INFO:__main__:Gerando URLs para o período: 2024-09
INFO:__main__:1 URLs geradas.
INFO:__main__:Baixando https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Simples.zip (Tentativa 1/15)
INFO:__main__:Download concluído: ./temp\Simples.zip
INFO:__main__:Descompactando ./temp\Simples.zip para ./output\Simples
INFO:__main__:Arquivo existente ./output\Simples\Simples.csv removido.
INFO:__main__:Arquivo descompactado com sucesso: ./output\Simples\Simples.csv
INFO:__main__:Arquivo zip ./temp\Simples.zip removido após descompactação.
INFO:__main__:Arquivo https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Simples.zip foi descompactado com sucesso para ./output


Result for https://dadosabertos.rfb.gov.br/CNPJ/dados_abertos_cnpj/2024-09/Simples.zip: Success


## Paralelizando com Spark

In [ ]:
api = ReceitaCNPJApi()
# Defina a função de download e descompactação
def download_and_unzip_spark(url):
    api = ReceitaCNPJApi()
    return api.download_and_unzip(url)

# Crie um RDD das URLs
x = api.lista_urls_receita()
urls = x[22:24]
urls_rdd = spark.sparkContext.parallelize(urls)

In [ ]:
# Faça o download e descompacte as URLs em paralelo usando o Spark
results = urls_rdd.map(download_and_unzip_spark).collect()

for result in results:
    print(f"Result for {result[0]}: {result[1]}")

# CLasse TL Receita

In [8]:
# ======================
# Bibliotecas padrão para utilidades básicas e manipulação de arquivos:
# ======================
import os
import sys
import time
import zipfile
import glob
from datetime import datetime, timedelta

# ======================
# Registros de logs:
# ======================
import logging

# ======================
# Requisições e manipulação de conteúdo web:
# ======================
import requests
from bs4 import BeautifulSoup

# ======================
# Utilidades e manipulação de dados:
# ======================
from collections import Counter
import chardet
import string
import pandas as pd
import gzip
import shutil
# import pycep_correios  # Descomente se necessário
import brazilcep

# ======================
# Manipulação de datas:
# ======================
from dateutil.relativedelta import relativedelta

# ======================
# Criptografia:
# ======================
import secrets
import base64
# from Crypto.Cipher import AES
# from Crypto.Random import get_random_bytes

# ======================
# Multitarefas:
# ======================
from concurrent.futures import ThreadPoolExecutor

# ======================
# Spark:
# ======================
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import (
    regexp_replace,
    when,
    length,
    to_date,
    upper,
    lower,
    col,
    udf,
    split,
    explode,
    coalesce,
    concat_ws,
    concat,
    lit,
    broadcast,
    regexp_extract,
    expr
)
import pyspark.sql.functions as F
from functools import reduce

# ======================
# Geolocalização:
# ======================
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Configuração do Logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


In [9]:
import os
import logging
import glob
import chardet
import requests
import gzip
import shutil
import secrets
from functools import reduce
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    udf, broadcast, col, upper, lower, when, regexp_extract, 
    regexp_replace, concat_ws, concat, lit, to_date, explode, 
    split, expr, length
)
from pyspark.sql.types import (
    StructType, StructField, StringType, IntegerType, FloatType
)
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

# Configuração do Logger
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class ReceitaLT:
    """
    A classe `ReceitaLT` facilita a manipulação e análise de dados da Receita Federal do Brasil.

    Atributos:
        spark (SparkSession): Sessão Spark para manipulação de dataframes.
        logger (Logger): Logger para capturar e exibir logs.
        
    Atributos estáticos:
        - estabelecimentos: Schema para dados de estabelecimentos.
        - empresas: Schema para dados das empresas.
        - municipios: Schema para municípios.
        - cnaes: Schema para CNAEs.
        - paises: Schema para países.
        - qualificacoes: Schema para qualificações.
        - socios: Schema para sócios.
        - simples: Schema para opções do Simples Nacional.
        - naturezas: Schema para naturezas jurídicas.
        - motivos: Schema para motivos de situações cadastrais.
        - dic_provedor: Dicionário para correção de nomes de provedores de email.
        
    Métodos:
        detect_encoding(file_pattern_or_path, num_bytes=10000): Detecta a codificação do arquivo ou arquivos fornecidos.

    Uso:
        1. Instancie a classe com uma sessão Spark.
        2. Utilize os schemas estáticos para leitura de arquivos.
        3. Use o método `detect_encoding` para determinar a codificação de arquivos antes de lê-los.
        
    Exemplo:
        from pyspark.sql import SparkSession
        
        spark_session = SparkSession.builder.appName("MyApp").getOrCreate()
        receita_helper = ReceitaLT(spark_session)
        encodings = receita_helper.detect_encoding("path/to/datafile.csv")
        df = spark_session.read.csv("path/to/datafile.csv", schema=ReceitaLT.empresas, encoding=encodings["path/to/datafile.csv"])
    """

    # Definição dos Schemas
    estabelecimentos = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("CNPJ_ORDEM", StringType(), nullable=True),
        StructField("CNPJ_DV", StringType(), nullable=True),
        StructField("MATRIZ_FILIAL", StringType(), nullable=True),
        StructField("NOME_FANTASIA", StringType(), nullable=True),
        StructField("SIT_CADASTRAL", IntegerType(), nullable=True),
        StructField("DT_SIT_CADASTRAL", StringType(), nullable=True),
        StructField("MOTIVO_CADASTRAL", StringType(), nullable=True),
        StructField("NOME_CIDADE_EXTERIOR", StringType(), nullable=True),
        StructField("PAIS", StringType(), nullable=True),
        StructField("DT_INICIO_ATIVIDADE", StringType(), nullable=True),
        StructField("CNAE_1", StringType(), nullable=True),
        StructField("CNAE_2", StringType(), nullable=True),
        StructField("TIPO_LOUGRADOURO", StringType(), nullable=True),
        StructField("LOGRADOURO", StringType(), nullable=True),
        StructField("NUMERO", IntegerType(), nullable=True),
        StructField("COMPLEMENTO", StringType(), nullable=True),
        StructField("BAIRRO", StringType(), nullable=True),
        StructField("CEP", IntegerType(), nullable=True),
        StructField("UF", StringType(), nullable=True),
        StructField("MUNICIPIO", StringType(), nullable=True),
        StructField("DDD1", StringType(), nullable=True),
        StructField("TEL1", StringType(), nullable=True),
        StructField("DDD2", StringType(), nullable=True),
        StructField("TEL2", StringType(), nullable=True),
        StructField("DDD_FAX", IntegerType(), nullable=True),
        StructField("FAX", IntegerType(), nullable=True),
        StructField("EMAIL", StringType(), nullable=True),
        StructField("SIT_ESPECIAL", StringType(), nullable=True),
        StructField("DT_SIT_ESPECIAL", StringType(), nullable=True)
    ])

    empresas = StructType([
        StructField("CNPJ", StringType(), nullable=True),
        StructField("NOME_EMPRESA", StringType(), nullable=True),
        StructField("COD_NAT_JURICA", StringType(), nullable=True),
        StructField("QUALIF_RESPONVAVEL", StringType(), nullable=True),
        StructField("CAP_SOCIAL", StringType(), nullable=True),
        StructField("PORTE", StringType(), nullable=True),
        StructField("ENTE_FEDERATIVO", StringType(), nullable=True)
    ])

    municipios = StructType([
        StructField("ID_MUNICPIO", StringType(), nullable=True),
        StructField("MUNICIPIO", StringType(), nullable=True)
    ])

    cnaes = StructType([
        StructField("COD_CNAE", StringType(), nullable=True),
        StructField("CNAE", StringType(), nullable=True)
    ])
    
    paises = StructType([
        StructField("COD_PAIS", StringType(), nullable=True),
        StructField("NM_PAIS", StringType(), nullable=True)
    ])
    
    qualificacoes = StructType([
        StructField("COD_QUALIFICACAO", StringType(), nullable=True),
        StructField("NM_QUALIFICACAO", StringType(), nullable=True)
    ])

    socios = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("IDENTIFICADOR_SOCIO", IntegerType(), nullable=True),
        StructField("NOME_SOCIO_RAZAO_SOCIAL", StringType(), nullable=True),
        StructField("CNPJ_CPF_SOCIO", StringType(), nullable=True),
        StructField("QUALIFICAÇAO_SOCIO", StringType(), nullable=True),
        StructField("DATA_ENTRADA_SOCIEDADE", StringType(), nullable=True),
        StructField("PAIS", StringType(), nullable=True),
        StructField("REPRESENTANTE_LEGAL", StringType(), nullable=True),
        StructField("NOME_REPRESENTANTE", StringType(), nullable=True),
        StructField("QUALIFICACAO_REPRESENTANTE_LEGAL", StringType(), nullable=True),
        StructField("FAIXA_ETARIA", StringType(), nullable=True)
    ])

    simples = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("OPCAO_PELO_SIMPLES", StringType(), nullable=True),
        StructField("DATA_OPCAO_PELO_SIMPLES", StringType(), nullable=True),
        StructField("DATA_EXCLUSAO_SIMPLES", StringType(), nullable=True),
        StructField("OPÇAO_PELO_MEI", StringType(), nullable=True),
        StructField("DATA_OPCAO_PELO_MEI", StringType(), nullable=True),
        StructField("DATA_EXCLUSAO_MEI", StringType(), nullable=True)
    ])

    naturezas = StructType([
        StructField("COD_NAT_JURICA", StringType(), nullable=True),
        StructField("NAT_JURICA", StringType(), nullable=True)
    ])
    
    motivos = StructType([
        StructField("COD_MOTIVO", StringType(), nullable=True),
        StructField("NM_MOTIVO", StringType(), nullable=True)
    ])

    dic_provedor = {
        # ... (seu dicionário completo aqui)
        # Certifique-se de que o dicionário esteja correto e sem duplicatas
    }

    def __init__(self, spark: SparkSession):
        """
        Inicializa a classe ReceitaLT.
        
        Parâmetros:
        spark (SparkSession): Uma sessão Spark ativa.
        """
        self.spark = spark
        self.logger = logging.getLogger(__name__)
        # A configuração do logger já foi feita no início do script

    @staticmethod
    def geocode_address(address):
        """
        Geocodifica um endereço, convertendo-o em coordenadas de latitude e longitude.

        Parâmetros:
            address (str): Endereço a ser geocodificado.

        Retorna:
            tuple: Um par contendo a latitude e a longitude do endereço fornecido. 
                   Se o endereço não puder ser geocodificado, retorna (None, None).

        Exemplo:
            lat, lon = geocode_address("1600 Amphitheatre Parkway, Mountain View, CA")

        Notas:
            - Usa o serviço Nominatim para a geocodificação.
            - Incorpora um limitador de taxa para garantir que não excedamos os limites de requisições por segundo 
              do serviço.
        """
        geolocator = Nominatim(user_agent="CNPJ_GEOLOCATION")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        location = geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)

    @staticmethod
    def geocode_udf(address):
        """
        UDF do Spark para geocodificar um endereço dentro de um DataFrame.

        Parâmetros:
            address (str): Endereço a ser geocodificado.

        Retorna:
            dict: Dicionário contendo a 'latitude' e a 'longitude' do endereço fornecido.
                  Se o endereço não puder ser geocodificado, os valores serão None.

        Exemplo:
            df.withColumn("location", ReceitaLT.geocode_udf(df["address"]))

        Notas:
            - Esta UDF encapsula a função `geocode_address`.
            - Retorna um tipo de dado complexo (Struct) com dois campos: 'latitude' e 'longitude'.
        """
        lat, lon = ReceitaLT.geocode_address(address)
        return {"latitude": lat, "longitude": lon}

    # Definição do Schema para a UDF
    geocode_schema = StructType([
        StructField("latitude", FloatType(), nullable=True),
        StructField("longitude", FloatType(), nullable=True)
    ])

    # Criação do UDF com o schema definido
    geocode_udf = udf(geocode_udf, geocode_schema)

    @staticmethod
    def detect_encoding(file_pattern_or_path, num_bytes=10000):
        """
        Detecta a codificação do arquivo ou arquivos fornecidos.
        
        Parâmetros:
            file_pattern_or_path (str): Caminho ou padrão do arquivo para detecção.
            num_bytes (int, opcional): Número de bytes para ler para a detecção. Padrão é 10000.
        
        Retorna:
            dict: Dicionário com caminho do arquivo como chave e codificação detectada como valor.
        """
        files = glob.glob(file_pattern_or_path)
        encodings = {}
        for file_path in files:
            with open(file_path, 'rb') as f:
                rawdata = f.read(num_bytes)
                encodings[file_path] = chardet.detect(rawdata)["encoding"]
        return encodings

    def read_data(self, schema_name, base_path=None):
        """
        Lê dados de vários arquivos CSV de acordo com o esquema e caminho base fornecidos, consolidando-os 
        em um único DataFrame do Spark.

        Parâmetros:
            schema_name (str): Nome do esquema a ser usado para a leitura dos arquivos.
                               Deve ser uma das chaves do dicionário `schemas`.

            base_path (str, opcional): Caminho base dos arquivos CSV.
                                       Se não for fornecido, ele tentará buscar da variável de ambiente 'BASE_PATH'.
                                       Caso não encontre, o padrão "./output" será utilizado.

        Retorna:
            DataFrame: DataFrame do Spark contendo os dados consolidados dos arquivos CSV.

        Exceções:
            Pode lançar uma exceção se o arquivo não estiver presente no caminho especificado ou
            se houver problemas de codificação ao ler o arquivo.

        Exemplo:
            receita_helper = ReceitaLT(spark_session)
            df = receita_helper.read_data("estabelecimentos", "/path/to/csv/files")

        Notas:
            - A função primeiro detecta a codificação dos arquivos antes de lê-los para garantir que 
              eles sejam lidos corretamente.
            - A função lida com múltiplos arquivos CSV e os une em um único DataFrame.
            - O formato de arquivo assumido é CSV com delimitador ";", sem cabeçalho e com aspas para delimitar campos.
        """
        schemas = {
            "estabelecimentos": self.estabelecimentos,
            "empresas": self.empresas,
            "municipios": self.municipios,
            "cnaes": self.cnaes,
            "socios": self.socios,
            "simples": self.simples,
            "naturezas": self.naturezas,
            "qualificacoes": self.qualificacoes,
            "motivos": self.motivos,
            "paises": self.paises
        }

        # Se o base_path não for fornecido, pegar da variável de ambiente ou usar um padrão.
        if not base_path:
            base_path = os.environ.get('BASE_PATH', "./output")

        if schema_name not in schemas:
            raise ValueError(f"Schema '{schema_name}' não está definido.")

        # Definir o padrão do caminho dos arquivos
        if schema_name in ['estabelecimentos', 'empresas', 'socios']:
            file_location_pattern = os.path.join(base_path, schema_name.capitalize(), '*.csv')
        else:
            file_location_pattern = os.path.join(base_path, schema_name.capitalize(), f"{schema_name.capitalize()}.csv")

        # Detectar codificações
        encodings = self.detect_encoding(file_location_pattern)
        self.logger.info(f"Detected encodings: {encodings}")

        # Agora, vamos ler cada arquivo com sua codificação correta e armazenar em uma lista de DataFrames
        dfs = []
        for file_location, encoding in encodings.items():
            try:
                df = (self.spark.read.format("csv")
                      .option("sep", ";")
                      .option("header", "false")
                      .option('quote', '"')
                      .option("escape", '"')
                      .option("encoding", encoding)
                      .schema(schemas[schema_name])
                      .load(file_location))
                dfs.append(df)
                self.logger.info(f"Arquivo lido com sucesso: {file_location}")
            except Exception as e:
                self.logger.error(f"Erro ao ler o arquivo {file_location}: {e}")
                raise e

        # Unir todos os DataFrames em um único DataFrame
        if dfs:
            final_df = reduce(lambda a, b: a.union(b), dfs)
        else:
            final_df = self.spark.createDataFrame([], schemas[schema_name])

        return final_df

    def save_data(self, df, path, num_partitions=1, file_format="parquet"):
        """
        Salva o DataFrame no caminho especificado.
        
        Parâmetros:
            df (DataFrame): DataFrame a ser salvo.
            path (str): Caminho de destino.
            num_partitions (int, opcional): Número de partições para salvar os dados (padrão é 1).
            file_format (str, opcional): Formato do arquivo para salvar os dados (padrão é "parquet").
        """
        # Reparticionar o DataFrame conforme a entrada do usuário
        df = df.repartition(num_partitions)
        
        # Salvar o DataFrame no caminho e formato especificados
        df.write.mode('overwrite').format(file_format).save(path)
        self.logger.info(f"DataFrame salvo com sucesso em {path} no formato {file_format}.")

    @staticmethod
    def download_nomes(save_base_path="./output/nomes"):
        """
        Baixa e extrai o arquivo nomes.csv.gz do dataset genero-nomes no Brasil.io.

        Parâmetros:
            save_base_path (str, opcional): Caminho base onde o arquivo será salvo. O padrão é './output/nomes'.

        Descrição:
            - Cria o diretório de salvamento se ele não existir.
            - Baixa o arquivo nomes.csv.gz da URL especificada.
            - Extrai o conteúdo do arquivo .gz.
            - Remove o arquivo .gz original, mantendo apenas o arquivo CSV extraído.

        Notas:
            - Esta função usa a biblioteca `requests` para baixar o arquivo.
            - A função verifica se a resposta do servidor é 200 (sucesso) antes de baixar o arquivo.
            - O arquivo .gz é extraído usando a biblioteca `gzip`.
        """
        import gzip  # Certifique-se de importar gzip
        os.makedirs(save_base_path, exist_ok=True)
        url = "https://data.brasil.io/dataset/genero-nomes/nomes.csv.gz"
        file_name = os.path.basename(url)
        file_path = os.path.join(save_base_path, file_name)
        extracted_file_path = os.path.join(save_base_path, file_name[:-3])  # remove .gz

        # Baixe o arquivo
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=128):
                    file.write(chunk)
            logger.info(f"Arquivo baixado com sucesso: {file_path}")
        else:
            logger.error(f"Falha ao baixar {url}. Status code: {response.status_code}")
            return

        # Extraia o arquivo
        try:
            with gzip.open(file_path, 'rb') as f_in:
                with open(extracted_file_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            logger.info(f"Arquivo extraído com sucesso: {extracted_file_path}")
        except Exception as e:
            logger.error(f"Erro ao extrair o arquivo {file_path}: {e}")
            raise e

        # Apague o arquivo .gz
        os.remove(file_path)
        logger.info(f"Arquivo .gz removido: {file_path}")

    def process_estabelecimentos(self, df):
        """
        Processa e enriquece o DataFrame de estabelecimentos com informações adicionais e transformações.

        Parâmetros:
            df (DataFrame): DataFrame inicial contendo informações de estabelecimentos.

        Retorna:
            DataFrame: DataFrame processado e enriquecido com novas colunas e informações.

        Descrição:
            - Lê dataframes adicionais relacionados a países, municípios, cnaes e motivos.
            - Realiza renomeações de colunas para facilitar junções.
            - Enriquece o dataframe com informações de motivos, cnaes, municípios e países.
            - Processa colunas de e-mail, separando provedores e corrigindo valores.
            - Converte colunas de data de string para formato de data.
            - Deriva colunas de ano e mês a partir de datas.
            - Processa e deriva novas colunas com base em mapeamentos para situação cadastral e tipo de estabelecimento.
            - Valida endereços de e-mail usando expressões regulares.
            - Combina informações de endereço para formar uma coluna completa de endereço.
            - Utiliza a função de geocodificação para obter coordenadas com base no endereço e, em caso de falha, com base no CEP.
            - Realiza correções na coluna de provedor de e-mail usando um dicionário de mapeamento.

        Notas:
            - Esta função faz uso intensivo das operações de DataFrame do PySpark.
            - Dependências: A função depende de outras funções e UDFs, como 'geocode_udf', bem como de variáveis de instância, como 'dic_provedor'.
        """
        from pyspark.sql.functions import broadcast  # Certifique-se de importar broadcast

        # Leitura dos DataFrames auxiliares
        df_pais = self.read_data(schema_name='paises')
        df_mun = self.read_data(schema_name='municipios')
        df_cnaes = self.read_data(schema_name='cnaes')
        df_motivos = self.read_data(schema_name='motivos')
        
        # Renomear colunas para facilitar as junções
        df = df.withColumnRenamed("CNAE_1", "COD_CNAE") \
               .withColumnRenamed("MUNICIPIO", "ID_MUNICPIO") \
               .withColumnRenamed("PAIS", "COD_PAIS") \
               .withColumnRenamed("MOTIVO_CADASTRAL", "COD_MOTIVO") 
        
        # Realizar junções com os DataFrames auxiliares
        df = df.join(broadcast(df_motivos), "COD_MOTIVO", "left").drop("COD_MOTIVO")
        df = df.join(broadcast(df_cnaes), "COD_CNAE", "left").drop("COD_CNAE")
        df = df.join(broadcast(df_mun), "ID_MUNICPIO", "left").drop("ID_MUNICPIO")
        df = df.join(broadcast(df_pais), "COD_PAIS", "left").drop("COD_PAIS")
        
        # Tratamento da coluna PROVEDOR
        df = df.withColumn("PROVEDOR", regexp_extract("EMAIL", "(?<=@)[^.]+(?=\\.)", 0))
        df = df.withColumn("PROVEDOR", upper(col("PROVEDOR")))
        df = df.withColumn("EMAIL", lower(col("EMAIL")))

        # Convertendo colunas de data
        df = df.withColumn("DT_SIT_CADASTRAL", to_date(col('DT_SIT_CADASTRAL'), "yyyyMMdd")) \
               .withColumn("DT_INICIO_ATIVIDADE", to_date(col('DT_INICIO_ATIVIDADE'), "yyyyMMdd")) \
               .withColumn("DT_SIT_ESPECIAL", to_date(col('DT_SIT_ESPECIAL'), "yyyyMMdd"))
        
        # Derivando colunas de ano e mês
        df = df.withColumn("ano_cadastro", F.year('DT_INICIO_ATIVIDADE')) \
               .withColumn("mes_cadastro", F.month('DT_INICIO_ATIVIDADE')) \
               .withColumn("ano_sit_cadastral", F.year('DT_SIT_CADASTRAL')) \
               .withColumn("mes_sit_cadastral", F.month('DT_SIT_CADASTRAL'))
        
        # Mapeamento para Situação Cadastral
        mapping_sit_cadastral = {1: 'NULA', 2: 'ATIVA', 3: 'SUSPENSA', 4: 'INAPTA', 8: 'BAIXADA'}
        
        # Criando a coluna NM_SIT_CADASTRAL
        df = df.withColumn("NM_SIT_CADASTRAL", 
                           when(col("SIT_CADASTRAL").isin(list(mapping_sit_cadastral.keys())), col("SIT_CADASTRAL")).otherwise(None))
        
        for key, value in mapping_sit_cadastral.items():
            df = df.withColumn("NM_SIT_CADASTRAL", 
                               when(col("SIT_CADASTRAL") == key, value).otherwise(col("NM_SIT_CADASTRAL")))
        
        # Validação de e-mails
        email_pattern = r'^\S+@\S+\.\S+$'  # Padrão simples de endereço de e-mail
        df = df.withColumn("valid_email", regexp_extract(col("EMAIL"), email_pattern, 0))
        
        # Mapeamento para Tipo de Estabelecimento
        mapping_tipo = {1: 'MATRIZ', 2: 'FILIAL'}
        
        df = df.withColumn("NM_MATRIZ_FILIAL", 
                           when(col("MATRIZ_FILIAL").isin(list(mapping_tipo.keys())), col("MATRIZ_FILIAL")).otherwise(None))
        
        for key, value in mapping_tipo.items():
            df = df.withColumn("NM_MATRIZ_FILIAL", 
                               when(col("MATRIZ_FILIAL") == key, value).otherwise(col("NM_MATRIZ_FILIAL")))
        
        # Criando a coluna ENDERECO_COMPLETO
        df = df.withColumn("ENDERECO_COMPLETO",
                           concat_ws(", ",
                                     concat(col("TIPO_LOUGRADOURO"), lit(" "), col("LOGRADOURO")),
                                     col("NUMERO"),
                                     concat_ws(" - ", col("MUNICIPIO"), col("UF"))))
        
        # Adicionando a lógica de geocodificação
        df = df.withColumn("COORDENADAS", ReceitaLT.geocode_udf(col("ENDERECO_COMPLETO")))
        
        # Caso a geocodificação falhe, tentar com o CEP
        df = df.withColumn("COORDENADAS",
                           when((col("COORDENADAS.latitude").isNull()) & (col("COORDENADAS.longitude").isNull()),
                                ReceitaLT.geocode_udf(col("CEP"))).otherwise(col("COORDENADAS")))
        
        # Correção da coluna PROVEDOR
        df = df.replace(self.dic_provedor, subset=['PROVEDOR'])
        
        # Transformação das keys e values do dicionário em lowercase
        dic_prov_lower = {k.lower(): str(v).lower() for k, v in self.dic_provedor.items()}
        
        # Correção dos provedores na coluna EMAIL
        replace_expr = reduce(
            lambda a, b: regexp_replace(a, rf"\b{b[0]}\b", b[1]),
            dic_prov_lower.items(),
            col("valid_email")
        )
        
        df = df.withColumn("valid_email", replace_expr) \
               .withColumnRenamed("valid_email", "VALID_EMAIL")
        
        return df

    def process_empresas(self, df):
        """
        Processa e enriquece o DataFrame de empresas com informações adicionais e transformações.

        Parâmetros:
            df (DataFrame): DataFrame inicial contendo informações de empresas.

        Retorna:
            DataFrame: DataFrame processado e enriquecido com novas colunas e informações.

        Descrição:
            - Lê dataframes adicionais relacionados a naturezas jurídicas e qualificações.
            - Realiza renomeação de colunas para facilitar junções.
            - Enriquece o dataframe com informações de naturezas jurídicas e qualificações.
            - Processa a coluna 'NOME_EMPRESA' para extrair informações potenciais de CPF.
            - Deriva uma nova coluna baseada no porte da empresa, usando um mapeamento predefinido.
            - Determina a probabilidade de um valor ser um CPF válido com base em seu comprimento.
            - Criptografa possíveis valores de CPF usando AES e os armazena em uma nova coluna 'CPF_CRIPTOGRAFADO', enquanto remove a coluna original 'CPF'.

        Notas:
            - Esta função faz uso intensivo das operações de DataFrame do PySpark.
            - O valor de criptografia (secret_key) é gerado dinamicamente a cada chamada da função. Portanto, cada execução resultará em valores de 'CPF_CRIPTOGRAFADO' diferentes para os mesmos CPFs.
            - O método AES usado aqui é 'ECB', que não é considerado seguro para muitos casos de uso devido à falta de vetor de inicialização (IV). A utilização deste modo deve ser revista se a segurança for uma preocupação.
        """
        from pyspark.sql.functions import expr

        # Leitura dos DataFrames auxiliares
        df_nat = self.read_data(schema_name='naturezas')
        df_qual = self.read_data(schema_name='qualificacoes')
        
        # Renomear coluna para facilitar a junção
        df = df.withColumnRenamed("QUALIF_RESPONVAVEL", "COD_QUALIFICACAO")
        
        # Realizar junções com os DataFrames auxiliares
        df = df.join(broadcast(df_nat), "COD_NAT_JURICA", "left").drop("COD_NAT_JURICA")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO")
               
        # Extração e tratamento da coluna CPF
        df = df.withColumn("CPF", regexp_replace(col("NOME_EMPRESA"), "[^0-9]", ""))
        df = df.withColumn("CPF", when(col("CPF") == "", None).otherwise(col("CPF")))
        df = df.withColumn('CPF_LEN', length('CPF'))
        
        # Mapeamento para Porte da Empresa
        mapping_porte = {
            0: 'NÃO INFORMADO',
            1: 'MICRO EMPRESA',
            3: 'EMPRESA DE PEQUENO PORTE',
            5: 'DEMAIS',
            8: 'BAIXADA'
        }

        # Criando a coluna NM_PORTE
        df = df.withColumn("NM_PORTE", 
                           when(col("PORTE").isin(list(mapping_porte.keys())), col("PORTE")).otherwise(None))
        
        for key, value in mapping_porte.items():
            df = df.withColumn("NM_PORTE", 
                               when(col("PORTE") == key, value).otherwise(col("NM_PORTE")))
        
        # Determinando a probabilidade de ser um CPF válido
        df = df.withColumn("PROBABILIDADE_DE_SER_CPF", 
                           when(col("CPF_LEN") == 11, "SIM").otherwise("NAO"))
        
        # Gerando uma chave secreta para criptografia
        secret_key = secrets.token_urlsafe(24)
        df = df.withColumn("CPF_CRIPTOGRAFADO", 
                           expr(f"base64(aes_encrypt(CPF, '{secret_key}', 'ECB', 'PKCS'))")) \
               .drop("CPF")
        
        return df

    def process_simples(self, df):
        """
        Processa o DataFrame relacionado ao regime tributário SIMPLES das empresas.

        Parâmetros:
            df (pyspark.sql.DataFrame): DataFrame inicial contendo informações relacionadas ao regime tributário SIMPLES.

        Retorna:
            pyspark.sql.DataFrame: DataFrame processado com colunas de data convertidas e apenas as colunas relevantes selecionadas.

        Descrição:
            - Converte colunas que representam datas do formato "yyyyMMdd" para o tipo data.
            - Seleciona apenas as colunas relevantes para o contexto, que são: 'CNPJ_BASICO', 'OPÇAO_PELO_MEI', 'DT_OPCAO_MEI', 'DT_EXCLUSAO_MEI', 'OPCAO_PELO_SIMPLES', 'DT_OPCAO_SIMPLES', e 'DT_EXCLUSAO_SIMPLES'.

        Notas:
            - Esta função assume que as colunas de data estão no formato "yyyyMMdd" e realiza a conversão para o tipo data.
            - As colunas de datas que são processadas incluem: DATA_OPCAO_PELO_SIMPLES, DATA_EXCLUSAO_SIMPLES, DATA_EXCLUSAO_MEI e DATA_OPCAO_PELO_MEI.
        """
        from pyspark.sql.functions import F

        df = df.withColumn("DT_OPCAO_SIMPLES", F.to_date(col('DATA_OPCAO_PELO_SIMPLES'), "yyyyMMdd")) \
               .withColumn("DT_EXCLUSAO_SIMPLES", to_date(col('DATA_EXCLUSAO_SIMPLES'), "yyyyMMdd")) \
               .withColumn("DT_EXCLUSAO_MEI", to_date(col('DATA_EXCLUSAO_MEI'), "yyyyMMdd")) \
               .withColumn("DT_OPCAO_MEI", to_date(col('DATA_OPCAO_PELO_MEI'), "yyyyMMdd")) \
               .select('CNPJ_BASICO','OPÇAO_PELO_MEI','DT_OPCAO_MEI','DT_EXCLUSAO_MEI',
                       'OPCAO_PELO_SIMPLES','DT_OPCAO_SIMPLES','DT_EXCLUSAO_SIMPLES')
        
        return df

    def process_mei(self, df, save_base_path="./output/nomes", file_name="nomes.csv"):
        """
        Processa um DataFrame referente a MEIs, realiza joins com dados adicionais de naturezas jurídicas,
        qualificações, e um conjunto de dados de nomes para extração e categorização de primeiro nome.

        Parâmetros:
            df (pyspark.sql.DataFrame): DataFrame inicial contendo dados sobre MEIs.
            save_base_path (str, opcional): Caminho onde o arquivo com dados de nomes foi extraído. Padrão é './output/nomes'.
            file_name (str, opcional): Nome do arquivo CSV contendo dados de nomes a ser lido. Padrão é 'nomes.csv'.

        Retorna:
            pyspark.sql.DataFrame: DataFrame processado após todas as transformações e joins.

        Descrição:
            1. Realiza join com DataFrames de 'naturezas' e 'qualificações'.
            2. Extração e manipulação de dados de CPF.
            3. Utiliza um dicionário para mapear e criar a coluna "NM_PORTE".
            4. Criptografa a coluna de CPF.
            5. Realiza filtragens baseado na probabilidade do nome ser um CPF válido.
            6. Lê um conjunto de dados de nomes e realiza o explode na coluna 'alternative_names'.
            7. Extrai o primeiro nome da coluna 'NOME_EMPRESA'.
            8. Realiza o join com o conjunto de dados de nomes para categorizar o primeiro nome.
            9. Retorna um DataFrame contendo informações relevantes após todas as transformações.
        """
        from pyspark.sql.functions import explode, split

        # Leitura dos DataFrames auxiliares
        df_nat = self.read_data(schema_name='naturezas')
        df_qual = self.read_data(schema_name='qualificacoes')
        
        # Renomear coluna para facilitar a junção
        df = df.withColumnRenamed("QUALIF_RESPONVAVEL", "COD_QUALIFICACAO")
        
        # Realizar junções com os DataFrames auxiliares
        df = df.join(broadcast(df_nat), "COD_NAT_JURICA", "left").drop("COD_NAT_JURICA")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO")
               
        # Extração e tratamento da coluna CPF
        df = df.withColumn("CPF", regexp_replace(col("NOME_EMPRESA"), "[^0-9]", ""))
        df = df.withColumn("CPF", when(col("CPF") == "", None).otherwise(col("CPF")))
        df = df.withColumn('CPF_LEN', length('CPF'))
        
        # Mapeamento para Porte da Empresa
        mapping_porte = {
            0: 'NÃO INFORMADO',
            1: 'MICRO EMPRESA',
            3: 'EMPRESA DE PEQUENO PORTE',
            5: 'DEMAIS',
            8: 'BAIXADA'
        }

        # Criando a coluna NM_PORTE
        df = df.withColumn("NM_PORTE", 
                           when(col("PORTE").isin(list(mapping_porte.keys())), col("PORTE")).otherwise(None))
        
        for key, value in mapping_porte.items():
            df = df.withColumn("NM_PORTE", 
                               when(col("PORTE") == key, value).otherwise(col("NM_PORTE")))
        
        # Determinando a probabilidade de ser um CPF válido
        df = df.withColumn("PROBABILIDADE_DE_SER_CPF", 
                           when(col("CPF_LEN") == 11, "SIM").otherwise("NAO"))
        
        # Gerando uma chave secreta para criptografia
        secret_key = secrets.token_urlsafe(24)
        df = df.withColumn("CPF_CRIPTOGRAFADO", 
                           expr(f"base64(aes_encrypt(CPF, '{secret_key}', 'ECB', 'PKCS'))")) \
               .drop("CPF")
        
        # Filtrar df_processado baseado na coluna PROBABILIDADE_DE_SER_CPF
        df_filter = df.filter(col('PROBABILIDADE_DE_SER_CPF') == 'SIM') \
                      .dropDuplicates(subset=['CPF_CRIPTOGRAFADO', 'NOME_EMPRESA'])
        
        # Caminho completo do arquivo de nomes
        file_path = os.path.join(save_base_path, file_name)
        
        # Ler o arquivo CSV de nomes
        df_csv = self.spark.read.csv(file_path, header=True, inferSchema=True)
        
        # Explodir a coluna alternative_names para múltiplas linhas
        df_csv = df_csv.withColumn("alternative_names", explode(split(coalesce(col("alternative_names"), col("first_name")), "\\|")))
        
        # Selecionar as colunas desejadas
        df_result = df_csv.select("alternative_names", "group_name", "ratio", "classification") \
                          .dropDuplicates(subset=['alternative_names'])
        
        # Extrair o primeiro nome da coluna NOME_EMPRESA
        df_filter = df_filter.withColumn("PRIMEIRO_NOME", split(col("NOME_EMPRESA"), " ")[0])
        
        # Fazer o join entre df_filter e df_result
        joined_df = df_filter.join(df_result, df_filter.PRIMEIRO_NOME == df_result.alternative_names, "left").dropDuplicates()
        
        # Selecionar as colunas finais
        joined_df = joined_df.select(
            'CNPJ', 'NOME_EMPRESA', 'CAP_SOCIAL', 'NM_PORTE', 'NAT_JURICA', 
            'ENTE_FEDERATIVO', 'NM_QUALIFICACAO', 'CPF_CRIPTOGRAFADO', 'CPF_LEN',
            'PROBABILIDADE_DE_SER_CPF', 'PRIMEIRO_NOME',
            col('group_name').alias('GRUPO_NOME'), 
            col('ratio').alias('PROBABILIDADE_CLASSIFICACAO'), 
            col('classification').alias('CLASSIFICACAO')
        ).dropDuplicates()
        
        return joined_df

    def process_socios(self, df, save_base_path="./output/nomes", file_name="nomes.csv"):
        """
        Processa um DataFrame referente a sócios, realiza joins com dados adicionais de países, qualificações, 
        e um conjunto de dados de nomes para extração e categorização de primeiro nome.

        Parâmetros:
            df (pyspark.sql.DataFrame): DataFrame inicial contendo dados sobre sócios.
            save_base_path (str, opcional): Caminho onde o arquivo com dados de nomes foi extraído. Padrão é './output/nomes'.
            file_name (str, opcional): Nome do arquivo CSV contendo dados de nomes a ser lido. Padrão é 'nomes.csv'.

        Retorna:
            pyspark.sql.DataFrame: DataFrame processado após todas as transformações e joins.

        Descrição:
            1. Realiza join com DataFrames de 'países'.
            2. Usa mapeamentos para criar colunas "NM_FAIXA_ETARIA" e "NM_IDENTIFICADOR_SOCIO".
            3. Renomeia e realiza join com DataFrame de qualificações para obter descrições das qualificações.
            4. Converte coluna de data "DATA_ENTRADA_SOCIEDADE" para o formato desejado.
            5. Lê um conjunto de dados de nomes e realiza o explode na coluna 'alternative_names'.
            6. Extrai o primeiro nome da coluna 'NOME_SOCIO_RAZAO_SOCIAL'.
            7. Realiza o join entre o DataFrame processado e o conjunto de dados de nomes para categorizar o primeiro nome.
            8. Retorna um DataFrame contendo informações relevantes após todas as transformações.
        """
        from pyspark.sql.functions import explode, split

        # Leitura dos DataFrames auxiliares
        df_pais = self.read_data(schema_name='paises')
        df = df.withColumnRenamed("PAIS", "COD_PAIS")
        df = df.join(broadcast(df_pais), "COD_PAIS", "left").drop("COD_PAIS")
        
        # Mapeamento de códigos para faixas etárias.
        mapping_faixa_etaria = {
            1: '0 a 12 anos',
            2: '13 a 20 anos',
            3: '21 a 30 anos',
            4: '31 a 40 anos',
            5: '41 a 50 anos',
            6: '51 a 60 anos',
            7: '61 a 70 anos',
            8: '71 a 80 anos',
            9: 'maiores de 80 anos',
            0: 'NA'
        }
        
        # Mapeamento de códigos para identificador de sócio.
        mapping_id_socio = {
            1: 'PESSOA JURIDICA',
            2: 'PESSOA FISICA',
            3: 'ESTRANGEIRO'
        }
        
        # Leitura do DataFrame de qualificações
        df_qual = self.read_data(schema_name='qualificacoes')
        
        # Criando a coluna NM_FAIXA_ETARIA
        df = df.withColumn("NM_FAIXA_ETARIA", 
                           when(col("FAIXA_ETARIA").isin(list(mapping_faixa_etaria.keys())), col("FAIXA_ETARIA")).otherwise(None))
        
        for key, value in mapping_faixa_etaria.items():
            df = df.withColumn("NM_FAIXA_ETARIA", 
                               when(col("FAIXA_ETARIA") == key, value).otherwise(col("NM_FAIXA_ETARIA")))
        
        # Criando a coluna NM_IDENTIFICADOR_SOCIO
        df = df.withColumn("NM_IDENTIFICADOR_SOCIO", 
                           when(col("IDENTIFICADOR_SOCIO").isin(list(mapping_id_socio.keys())), col("IDENTIFICADOR_SOCIO")).otherwise(None))
        
        for key, value in mapping_id_socio.items():
            df = df.withColumn("NM_IDENTIFICADOR_SOCIO", 
                               when(col("IDENTIFICADOR_SOCIO") == key, value).otherwise(col("NM_IDENTIFICADOR_SOCIO")))
        
        # Renomear e juntar com qualificações para obter descrições
        df = df.withColumnRenamed("QUALIFICACAO_REPRESENTANTE_LEGAL", "COD_QUALIFICACAO")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO") \
               .withColumnRenamed("NM_QUALIFICACAO", "NM_QUALIFICACAO_REPRESENTANTE_LEGAL")
        
        df = df.withColumnRenamed("QUALIFICAÇAO_SOCIO", "COD_QUALIFICACAO")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO") \
               .withColumnRenamed("NM_QUALIFICACAO", "NM_QUALIFICAÇAO_SOCIO")
        
        # Conversão da coluna de data
        df = df.withColumn("DT_ENTRADA_SOCIEDADE", to_date(col('DATA_ENTRADA_SOCIEDADE'), "yyyyMMdd")) \
               .drop("DATA_ENTRADA_SOCIEDADE")
        
        # Leitura do arquivo CSV de nomes
        file_path = os.path.join(save_base_path, file_name)
        df_csv = self.spark.read.csv(file_path, header=True, inferSchema=True)
        
        # Explodir a coluna alternative_names para múltiplas linhas
        df_csv = df_csv.withColumn("alternative_name2", explode(split(col("alternative_names"), "\|")))
        
        # Selecionar as colunas desejadas
        df_result = df_csv.select("alternative_name2", "group_name", "ratio", "classification") \
                          .dropDuplicates(["alternative_name2"])
        
        # Extrair o primeiro nome da coluna NOME_SOCIO_RAZAO_SOCIAL
        df = df.withColumn("PRIMEIRO_NOME", split(col("NOME_SOCIO_RAZAO_SOCIAL"), " ")[0].alias("PRIMEIRO_NOME"))
        
        # Fazer o join entre df_filter e df_result
        joined_df = df.join(df_result, df.PRIMEIRO_NOME == df_result.alternative_name2, "left").dropDuplicates()
        
        # Selecionar as colunas finais
        joined_df = joined_df.select(
            'CNPJ_BASICO', 'NOME_SOCIO_RAZAO_SOCIAL', 'CNPJ_CPF_SOCIO', 'REPRESENTANTE_LEGAL',
            'NOME_REPRESENTANTE', 'NM_PAIS', 'NM_FAIXA_ETARIA', 'NM_IDENTIFICADOR_SOCIO',
            'NM_QUALIFICACAO_REPRESENTANTE_LEGAL', 'NM_QUALIFICAÇAO_SOCIO', 'DT_ENTRADA_SOCIEDADE',
            'PRIMEIRO_NOME', 
            col('ratio').alias('PROBABILIDADE_CLASSIFICACAO'),
            col('classification').alias('CLASSIFICACAO')
        ).dropDuplicates()
        
        return joined_df


In [14]:
class ReceitaLT:
    """
    A classe `ReceitaLT` facilita a manipulação e análise de dados da Receita Federal do Brasil.

    Atributos:
        spark (SparkSession): Sessão Spark para manipulação de dataframes.
        logger (Logger): Logger para capturar e exibir logs.

    Atributos estáticos:
        - estabelecimentos: Schema para dados de estabelecimentos.
        - empresas: Schema para dados das empresas.
        - municipios: Schema para municípios.
        - cnaes: Schema para CNAEs.
        - paises: Schema para países.
        - qualificacoes: Schema para qualificações.
        - socios: Schema para sócios.
        - simples: Schema para opções do Simples Nacional.
        - naturezas: Schema para naturezas jurídicas.
        - motivos: Schema para motivos de situações cadastrais.
        - dic_provedor: Dicionário para correção de nomes de provedores de email.

    Métodos:
        detect_encoding(file_pattern_or_path, num_bytes=10000): Detecta a codificação do arquivo ou arquivos fornecidos.

    Uso:
        1. Instancie a classe com uma sessão Spark.
        2. Utilize os schemas estáticos para leitura de arquivos.
        3. Use o método `detect_encoding` para determinar a codificação de arquivos antes de lê-los.

    Exemplo:
        from pyspark.sql import SparkSession

        spark_session = SparkSession.builder.appName("MyApp").getOrCreate()
        receita_helper = ReceitaLT(spark_session)
        encodings = receita_helper.detect_encoding("path/to/datafile.csv")
        df = spark_session.read.csv("path/to/datafile.csv", schema=ReceitaLT.empresas, encoding=encodings["path/to/datafile.csv"])
    """

    # Definição dos Schemas
    estabelecimentos = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("CNPJ_ORDEM", StringType(), nullable=True),
        StructField("CNPJ_DV", StringType(), nullable=True),
        StructField("MATRIZ_FILIAL", StringType(), nullable=True),
        StructField("NOME_FANTASIA", StringType(), nullable=True),
        StructField("SIT_CADASTRAL", IntegerType(), nullable=True),
        StructField("DT_SIT_CADASTRAL", StringType(), nullable=True),
        StructField("MOTIVO_CADASTRAL", StringType(), nullable=True),
        StructField("NOME_CIDADE_EXTERIOR", StringType(), nullable=True),
        StructField("PAIS", StringType(), nullable=True),
        StructField("DT_INICIO_ATIVIDADE", StringType(), nullable=True),
        StructField("CNAE_1", StringType(), nullable=True),
        StructField("CNAE_2", StringType(), nullable=True),
        StructField("TIPO_LOUGRADOURO", StringType(), nullable=True),
        StructField("LOGRADOURO", StringType(), nullable=True),
        StructField("NUMERO", IntegerType(), nullable=True),
        StructField("COMPLEMENTO", StringType(), nullable=True),
        StructField("BAIRRO", StringType(), nullable=True),
        StructField("CEP", IntegerType(), nullable=True),
        StructField("UF", StringType(), nullable=True),
        StructField("MUNICIPIO", StringType(), nullable=True),
        StructField("DDD1", StringType(), nullable=True),
        StructField("TEL1", StringType(), nullable=True),
        StructField("DDD2", StringType(), nullable=True),
        StructField("TEL2", StringType(), nullable=True),
        StructField("DDD_FAX", IntegerType(), nullable=True),
        StructField("FAX", IntegerType(), nullable=True),
        StructField("EMAIL", StringType(), nullable=True),
        StructField("SIT_ESPECIAL", StringType(), nullable=True),
        StructField("DT_SIT_ESPECIAL", StringType(), nullable=True)
    ])

    empresas = StructType([
        StructField("CNPJ", StringType(), nullable=True),
        StructField("NOME_EMPRESA", StringType(), nullable=True),
        StructField("COD_NAT_JURICA", StringType(), nullable=True),
        StructField("QUALIF_RESPONVAVEL", StringType(), nullable=True),
        StructField("CAP_SOCIAL", StringType(), nullable=True),
        StructField("PORTE", StringType(), nullable=True),
        StructField("ENTE_FEDERATIVO", StringType(), nullable=True)
    ])

    municipios = StructType([
        StructField("ID_MUNICPIO", StringType(), nullable=True),
        StructField("MUNICIPIO", StringType(), nullable=True)
    ])

    cnaes = StructType([
        StructField("COD_CNAE", StringType(), nullable=True),
        StructField("CNAE", StringType(), nullable=True)
    ])
    
    paises = StructType([
        StructField("COD_PAIS", StringType(), nullable=True),
        StructField("NM_PAIS", StringType(), nullable=True)
    ])
    
    qualificacoes = StructType([
        StructField("COD_QUALIFICACAO", StringType(), nullable=True),
        StructField("NM_QUALIFICACAO", StringType(), nullable=True)
    ])

    socios = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("IDENTIFICADOR_SOCIO", IntegerType(), nullable=True),
        StructField("NOME_SOCIO_RAZAO_SOCIAL", StringType(), nullable=True),
        StructField("CNPJ_CPF_SOCIO", StringType(), nullable=True),
        StructField("QUALIFICAÇAO_SOCIO", StringType(), nullable=True),
        StructField("DATA_ENTRADA_SOCIEDADE", StringType(), nullable=True),
        StructField("PAIS", StringType(), nullable=True),
        StructField("REPRESENTANTE_LEGAL", StringType(), nullable=True),
        StructField("NOME_REPRESENTANTE", StringType(), nullable=True),
        StructField("QUALIFICACAO_REPRESENTANTE_LEGAL", StringType(), nullable=True),
        StructField("FAIXA_ETARIA", StringType(), nullable=True)
    ])

    simples = StructType([
        StructField("CNPJ_BASICO", StringType(), nullable=True),
        StructField("OPCAO_PELO_SIMPLES", StringType(), nullable=True),
        StructField("DATA_OPCAO_PELO_SIMPLES", StringType(), nullable=True),
        StructField("DATA_EXCLUSAO_SIMPLES", StringType(), nullable=True),
        StructField("OPÇAO_PELO_MEI", StringType(), nullable=True),
        StructField("DATA_OPCAO_PELO_MEI", StringType(), nullable=True),
        StructField("DATA_EXCLUSAO_MEI", StringType(), nullable=True)
    ])

    naturezas = StructType([
        StructField("COD_NAT_JURICA", StringType(), nullable=True),
        StructField("NAT_JURICA", StringType(), nullable=True)
    ])
    
    motivos = StructType([
        StructField("COD_MOTIVO", StringType(), nullable=True),
        StructField("NM_MOTIVO", StringType(), nullable=True)
    ])

    dic_provedor = {
        'OUTLOOK': 'OUTLOOK', 'OUTIOOK': 'OUTLOOK', 'OUTLLOK': 'OUTLOOK', 'OUTLLOOK': 'OUTLOOK', 
        'OUTLOCK': 'OUTLOOK', 'OUTLOK': 'OUTLOOK', 'OUTLOKK': 'OUTLOOK', 'OUTLOOCK': 'OUTLOOK', 'OUTLOOK': 'OUTLOOK', 
        'OUTLOOKL': 'OUTLOOK', 'OUTLOOL': 'OUTLOOK', 'OUTLOOOK': 'OUTLOOK', 'OUTLUK': 'OUTLOOK', 'OUTOLOOK': 'OUTLOOK',
        'OUTOOK': 'OUTLOOK', 'OUTOOLK': 'OUTLOOK', 'OUTTLOOK': 'OUTLOOK', 'OUTULOOK': 'OUTLOOK', 'POP': 'POP',
        'PROTON': 'PROTONMAIL', 'PROTONMAIL': 'PROTONMAIL', 'PUTLOOK': 'OUTLOOK', 'R7': 'R7', 'ROCKETMAIL': 'ROCKETMAIL', 
        'ROCKTMAIL': 'ROCKETMAIL', 'ROTMAIL': 'HOTMAIL', 'SERCOMTEL': 'SERCOMTEL', 'SETELAGOASGML': 'GMAIL', 
        'SUPERIG': 'SUPERIG', 'TAHOO': 'YAHOO', 'TERRA': 'TERRA', 'TERRRA': 'TERRA', 'TMAIL': 'GMAIL', 
        'TVGLOBO': 'GLOBO', 'UAHOO': 'YAHOO', 'UAI': 'UAI', 'UFV': 'UFV', 'UNESP': 'UNESP', 'UNOCHAPECO': 'UNOCHAPECO', 
        'UO': 'UOL', 'UOL': 'UOL', 'UOTLOOK': 'OUTLOOK', 'UPF': 'UPF', 'USP': 'USP', 'UTLOOK': 'OUTLOOK', 'VELOXMAIL': 'VELOXMAIL',
        'WINDOWSLIVE': 'WINDOWSLIVE', 'YAAHOO': 'YAHOO', 'YAGOO': 'YAHOO', 'YAHAOO': 'YAHOO', 'YAHHO': 'YAHOO', 'YAHHOO': 'YAHOO', 
        'YAHO': 'YAHOO', 'YAHOO': 'YAHOO', 'YAHOOCOM': 'YAHOO', 'YAHOOL': 'YAHOO', 'YAHOOO': 'YAHOO', 'YAHOOU': 'YAHOO', 
        'YANHOO': 'YAHOO', 'YAOO': 'YAHOO', 'YAOOL': 'YAHOO', 'YAROO': 'YAHOO', 'YHAOO': 'YAHOO', 'YHOO': 'YAHOO', 'YMAIL': 'YMAIL', 
        'YOHOO': 'YAHOO', 'YOPMAIL': 'HOTMAIL', 'ZIPMAIL': 'ZIPMAIL', '_HOTMAIL': 'HOTMAIL', 'GMAUL': 'GMAIL','GMALE': 'GMAIL', 
        'GMAILE': 'GMAIL', 'GMILE': 'GMAIL', 'HOTMEL': 'HOTMAIL', 'HOTMELL': 'HOTMAIL','HOTMEAL': 'HOTMAIL', 'OUTLOKES': 'OUTLOOK', 
        'OTLOOKS': 'OUTLOOK', 'YAHU': 'YAHOO', 'YOHU': 'YAHOO', 'YAHUO': 'YAHOO', 'YAHEE': 'YAHOO', 'UOLL': 'UOL',
        'UOOL': 'UOL', 'UULL': 'UOL', 'ICLODUE': 'ICLOUD', 'ICLAWD': 'ICLOUD', 'ROCKEDMAIL': 'ROCKETMAIL', 'ROKETMAIL': 'ROCKETMAIL',
        'OUTLOKE': 'OUTLOOK', 'OUTLOOCKE': 'OUTLOOK', 'YAAHO': 'YAHOO', 'YAHOOE': 'YAHOO', 'YAHUE': 'YAHOO', 'HOTMILE': 'HOTMAIL', 'HOTMELE': 'HOTMAIL',
        'FACEBOKE': 'FACEBOOK', 'FACBOOK': 'FACEBOOK', 'FCEBOOK': 'FACEBOOK', 'BOLL': 'BOL', 'BOLLE': 'BOL', 'BULE': 'BOL', 'GLOBOE': 'GLOBO',
        'GLOBU': 'GLOBO', 'GMILE': 'GMAIL', 'MSNE': 'MSN', 'MSNN': 'MSN', 'ICLOOUD': 'ICLOUD', 'OUTLUKE': 'OUTLOOK', 'OUTLLOKE': 'OUTLOOK',
        'PROTONMIAL': 'PROTONMAIL', 'PROTONMALE': 'PROTONMAIL', 'PROTOMAIL': 'PROTONMAIL', 
        'OULOOKCOM': 'OUTLOOK', 'YAHCOM': 'YAHOO',  'YAHOCOM': 'YAHOO','GAMILCOM': 'GMAIL', 'GMALCOM': 'GMAIL',  
        'HOTMALCOM': 'HOTMAIL',  'HOTMILCOM': 'HOTMAIL', 'HOTMELCOM': 'HOTMAIL', 'ROCKMAIL': 'ROCKETMAIL', 
        'ROKMAIL': 'ROCKETMAIL', 'TERA': 'TERRA', 'TEERA': 'TERRA', 
        'FACBOOKCOM': 'FACEBOOK', 'FACEBOOKCOM': 'FACEBOOK', 'ICLOWD': 'ICLOUD', 'ICLOUND': 'ICLOUD', 
        'UOOLCOM': 'UOL', 'UOLLCOM': 'UOL', 'UOLCOMBR': 'UOL','LIVECOM': 'LIVE', 'LIVECOMBR': 'LIVE', 
        'GMAICOM': 'GMAIL',  'GMAILCOMBR': 'GMAIL',  'YAHOOBR': 'YAHOO', 
        'YAHOOOCOMBR': 'YAHOO', 'YAHOOOCOM': 'YAHOO', 'ZIPMAILE': 'ZIPMAIL', 'ZIPMAILL': 'ZIPMAIL',  
        'IBESTT': 'IBEST', 'IBESTE': 'IBEST'
    }

    def __init__(self, spark: SparkSession):
        """
        Inicializa a classe ReceitaLT.
        
        Parâmetros:
        spark (SparkSession): Uma sessão Spark ativa.
        """
        self.spark = spark
        self.logger = logging.getLogger(__name__)
        # A configuração do logger já foi feita no início do script

    @staticmethod
    def geocode_address(address):
        """
        Geocodifica um endereço, convertendo-o em coordenadas de latitude e longitude.

        Parâmetros:
            address (str): Endereço a ser geocodificado.

        Retorna:
            tuple: Um par contendo a latitude e a longitude do endereço fornecido. 
                   Se o endereço não puder ser geocodificado, retorna (None, None).

        Exemplo:
            lat, lon = geocode_address("1600 Amphitheatre Parkway, Mountain View, CA")

        Notas:
            - Usa o serviço Nominatim para a geocodificação.
            - Incorpora um limitador de taxa para garantir que não excedamos os limites de requisições por segundo 
              do serviço.
        """
        geolocator = Nominatim(user_agent="CNPJ_GEOLOCATION")
        geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
        location = geocode(address)
        if location:
            return (location.latitude, location.longitude)
        else:
            return (None, None)

    @staticmethod
    def geocode_udf(address):
        """
        UDF do Spark para geocodificar um endereço dentro de um DataFrame.

        Parâmetros:
            address (str): Endereço a ser geocodificado.

        Retorna:
            dict: Dicionário contendo a 'latitude' e a 'longitude' do endereço fornecido.
                  Se o endereço não puder ser geocodificado, os valores serão None.

        Exemplo:
            df.withColumn("location", ReceitaLT.geocode_udf(df["address"]))

        Notas:
            - Esta UDF encapsula a função `geocode_address`.
            - Retorna um tipo de dado complexo (Struct) com dois campos: 'latitude' e 'longitude'.
        """
        lat, lon = ReceitaLT.geocode_address(address)
        return {"latitude": lat, "longitude": lon}

    # Definição do Schema para a UDF
    geocode_schema = StructType([
        StructField("latitude", FloatType(), nullable=True),
        StructField("longitude", FloatType(), nullable=True)
    ])

    # Criação do UDF com o schema definido
    geocode_udf = udf(geocode_udf, geocode_schema)

    @staticmethod
    def detect_encoding(file_pattern_or_path, num_bytes=10000):
        """
        Detecta a codificação do arquivo ou arquivos fornecidos.
        
        Parâmetros:
            file_pattern_or_path (str): Caminho ou padrão do arquivo para detecção.
            num_bytes (int, opcional): Número de bytes para ler para a detecção. Padrão é 10000.
        
        Retorna:
            dict: Dicionário com caminho do arquivo como chave e codificação detectada como valor.
        """
        files = glob.glob(file_pattern_or_path)
        encodings = {}
        for file_path in files:
            with open(file_path, 'rb') as f:
                rawdata = f.read(num_bytes)
                encodings[file_path] = chardet.detect(rawdata)["encoding"]
        return encodings

    def read_data(self, schema_name, base_path=None):
        """
        Lê dados de vários arquivos CSV de acordo com o esquema e caminho base fornecidos, consolidando-os 
        em um único DataFrame do Spark.

        Parâmetros:
            schema_name (str): Nome do esquema a ser usado para a leitura dos arquivos.
                               Deve ser uma das chaves do dicionário `schemas`.

            base_path (str, opcional): Caminho base dos arquivos CSV.
                                       Se não for fornecido, ele tentará buscar da variável de ambiente 'BASE_PATH'.
                                       Caso não encontre, o padrão "./output" será utilizado.

        Retorna:
            DataFrame: DataFrame do Spark contendo os dados consolidados dos arquivos CSV.

        Exceções:
            Pode lançar uma exceção se o arquivo não estiver presente no caminho especificado ou
            se houver problemas de codificação ao ler o arquivo.

        Exemplo:
            receita_helper = ReceitaLT(spark_session)
            df = receita_helper.read_data("estabelecimentos", "/path/to/csv/files")

        Notas:
            - A função primeiro detecta a codificação dos arquivos antes de lê-los para garantir que 
              eles sejam lidos corretamente.
            - A função lida com múltiplos arquivos CSV e os une em um único DataFrame.
            - O formato de arquivo assumido é CSV com delimitador ";", sem cabeçalho e com aspas para delimitar campos.
        """
        schemas = {
            "estabelecimentos": self.estabelecimentos,
            "empresas": self.empresas,
            "municipios": self.municipios,
            "cnaes": self.cnaes,
            "socios": self.socios,
            "simples": self.simples,
            "naturezas": self.naturezas,
            "qualificacoes": self.qualificacoes,
            "motivos": self.motivos,
            "paises": self.paises
        }

        # Se o base_path não for fornecido, pegar da variável de ambiente ou usar um padrão.
        if not base_path:
            base_path = os.environ.get('BASE_PATH', "./output")

        if schema_name not in schemas:
            raise ValueError(f"Schema '{schema_name}' não está definido.")

        # Definir o padrão do caminho dos arquivos
        if schema_name in ['estabelecimentos', 'empresas', 'socios']:
            file_location_pattern = os.path.join(base_path, schema_name.capitalize(), '*.csv')
        else:
            file_location_pattern = os.path.join(base_path, schema_name.capitalize(), f"{schema_name.capitalize()}.csv")

        # Detectar codificações
        encodings = self.detect_encoding(file_location_pattern)
        self.logger.info(f"Detected encodings: {encodings}")

        # Agora, vamos ler cada arquivo com sua codificação correta e armazenar em uma lista de DataFrames
        dfs = []
        for file_location, encoding in encodings.items():
            try:
                df = (self.spark.read.format("csv")
                      .option("sep", ";")
                      .option("header", "false")
                      .option('quote', '"')
                      .option("escape", '"')
                      .option("encoding", encoding)
                      .schema(schemas[schema_name])
                      .load(file_location))
                dfs.append(df)
                self.logger.info(f"Arquivo lido com sucesso: {file_location}")
            except Exception as e:
                self.logger.error(f"Erro ao ler o arquivo {file_location}: {e}")
                raise e

        # Unir todos os DataFrames em um único DataFrame
        if dfs:
            final_df = reduce(lambda a, b: a.union(b), dfs)
        else:
            final_df = self.spark.createDataFrame([], schemas[schema_name])

        return final_df

    def save_data(self, df, path, num_partitions=16, file_format="parquet"):
        """
        Salva o DataFrame no caminho especificado.
        
        Parâmetros:
            df (DataFrame): DataFrame a ser salvo.
            path (str): Caminho de destino.
            num_partitions (int, opcional): Número de partições para salvar os dados (padrão é 1).
            file_format (str, opcional): Formato do arquivo para salvar os dados (padrão é "parquet").
        """
        # Reparticionar o DataFrame conforme a entrada do usuário
        df = df.repartition(num_partitions)
        
        # Salvar o DataFrame no caminho e formato especificados
        df.write.mode('overwrite').format(file_format).save(path)
        self.logger.info(f"DataFrame salvo com sucesso em {path} no formato {file_format}.")

    @staticmethod
    def download_nomes(save_base_path="./output/nomes"):
        """
        Baixa e extrai o arquivo nomes.csv.gz do dataset genero-nomes no Brasil.io.

        Parâmetros:
            save_base_path (str, opcional): Caminho base onde o arquivo será salvo. O padrão é './output/nomes'.

        Descrição:
            - Cria o diretório de salvamento se ele não existir.
            - Baixa o arquivo nomes.csv.gz da URL especificada.
            - Extrai o conteúdo do arquivo .gz.
            - Remove o arquivo .gz original, mantendo apenas o arquivo CSV extraído.

        Notas:
            - Esta função usa a biblioteca `requests` para baixar o arquivo.
            - A função verifica se a resposta do servidor é 200 (sucesso) antes de baixar o arquivo.
            - O arquivo .gz é extraído usando a biblioteca `gzip`.
        """
        os.makedirs(save_base_path, exist_ok=True)
        url = "https://data.brasil.io/dataset/genero-nomes/nomes.csv.gz"
        file_name = os.path.basename(url)
        file_path = os.path.join(save_base_path, file_name)
        extracted_file_path = os.path.join(save_base_path, file_name[:-3])  # remove .gz

        # Baixe o arquivo
        response = requests.get(url, stream=True)
        if response.status_code == 200:
            with open(file_path, 'wb') as file:
                for chunk in response.iter_content(chunk_size=128):
                    file.write(chunk)
            logger.info(f"Arquivo baixado com sucesso: {file_path}")
        else:
            logger.error(f"Falha ao baixar {url}. Status code: {response.status_code}")
            return

        # Extraia o arquivo
        try:
            with gzip.open(file_path, 'rb') as f_in:
                with open(extracted_file_path, 'wb') as f_out:
                    shutil.copyfileobj(f_in, f_out)
            logger.info(f"Arquivo extraído com sucesso: {extracted_file_path}")
        except Exception as e:
            logger.error(f"Erro ao extrair o arquivo {file_path}: {e}")
            raise e

        # Apague o arquivo .gz
        os.remove(file_path)
        logger.info(f"Arquivo .gz removido: {file_path}")

    def process_estabelecimentos(self, df):
        """
        Processa e enriquece o DataFrame de estabelecimentos com informações adicionais e transformações.

        Parâmetros:
            df (DataFrame): DataFrame inicial contendo informações de estabelecimentos.

        Retorna:
            DataFrame: DataFrame processado e enriquecido com novas colunas e informações.

        Descrição:
            - Lê dataframes adicionais relacionados a países, municípios, cnaes e motivos.
            - Realiza renomeações de colunas para facilitar junções.
            - Enriquece o dataframe com informações de motivos, cnaes, municípios e países.
            - Processa colunas de e-mail, separando provedores e corrigindo valores.
            - Converte colunas de data de string para formato de data.
            - Deriva colunas de ano e mês a partir de datas.
            - Processa e deriva novas colunas com base em mapeamentos para situação cadastral e tipo de estabelecimento.
            - Valida endereços de e-mail usando expressões regulares.
            - Combina informações de endereço para formar uma coluna completa de endereço.
            - Utiliza a função de geocodificação para obter coordenadas com base no endereço e, em caso de falha, com base no CEP.
            - Realiza correções na coluna de provedor de e-mail usando um dicionário de mapeamento.

        Notas:
            - Esta função faz uso intensivo das operações de DataFrame do PySpark.
            - Dependências: A função depende de outras funções e UDFs, como 'geocode_udf', bem como de variáveis de instância, como 'dic_provedor'.
        """
        # Leitura dos DataFrames auxiliares
        df_pais = self.read_data(schema_name='paises')
        df_mun = self.read_data(schema_name='municipios')
        df_cnaes = self.read_data(schema_name='cnaes')
        df_motivos = self.read_data(schema_name='motivos')
        
        # Renomear colunas para facilitar as junções
        df = df.withColumnRenamed("CNAE_1", "COD_CNAE") \
               .withColumnRenamed("MUNICIPIO", "ID_MUNICPIO") \
               .withColumnRenamed("PAIS", "COD_PAIS") \
               .withColumnRenamed("MOTIVO_CADASTRAL", "COD_MOTIVO") 
        
        # Realizar junções com os DataFrames auxiliares
        df = df.join(broadcast(df_motivos), "COD_MOTIVO", "left").drop("COD_MOTIVO")
        df = df.join(broadcast(df_cnaes), "COD_CNAE", "left").drop("COD_CNAE")
        df = df.join(broadcast(df_mun), "ID_MUNICPIO", "left").drop("ID_MUNICPIO")
        df = df.join(broadcast(df_pais), "COD_PAIS", "left").drop("COD_PAIS")
        
        # Tratamento da coluna PROVEDOR
        df = df.withColumn("PROVEDOR", regexp_extract("EMAIL", "(?<=@)[^.]+(?=\\.)", 0))
        df = df.withColumn("PROVEDOR", upper(col("PROVEDOR")))
        df = df.withColumn("EMAIL", lower(col("EMAIL")))

        # Convertendo colunas de data
        df = df.withColumn("DT_SIT_CADASTRAL", to_date(col('DT_SIT_CADASTRAL'), "yyyyMMdd")) \
               .withColumn("DT_INICIO_ATIVIDADE", to_date(col('DT_INICIO_ATIVIDADE'), "yyyyMMdd")) \
               .withColumn("DT_SIT_ESPECIAL", to_date(col('DT_SIT_ESPECIAL'), "yyyyMMdd"))
        
        # Derivando colunas de ano e mês
        df = df.withColumn("ano_cadastro", F.year('DT_INICIO_ATIVIDADE')) \
               .withColumn("mes_cadastro", F.month('DT_INICIO_ATIVIDADE')) \
               .withColumn("ano_sit_cadastral", F.year('DT_SIT_CADASTRAL')) \
               .withColumn("mes_sit_cadastral", F.month('DT_SIT_CADASTRAL'))
        
        # Mapeamento para Situação Cadastral
        mapping_sit_cadastral = {1: 'NULA', 2: 'ATIVA', 3: 'SUSPENSA', 4: 'INAPTA', 8: 'BAIXADA'}
        
        # Criando a coluna NM_SIT_CADASTRAL
        df = df.withColumn("NM_SIT_CADASTRAL", 
                           when(col("SIT_CADASTRAL").isin(list(mapping_sit_cadastral.keys())), col("SIT_CADASTRAL")).otherwise(None))
        
        for key, value in mapping_sit_cadastral.items():
            df = df.withColumn("NM_SIT_CADASTRAL", 
                               when(col("SIT_CADASTRAL") == key, value).otherwise(col("NM_SIT_CADASTRAL")))
        
        # Validação de e-mails
        email_pattern = r'^\S+@\S+\.\S+$'  # Padrão simples de endereço de e-mail
        df = df.withColumn("valid_email", regexp_extract(col("EMAIL"), email_pattern, 0))
        
        # Mapeamento para Tipo de Estabelecimento
        mapping_tipo = {1: 'MATRIZ', 2: 'FILIAL'}
        
        df = df.withColumn("NM_MATRIZ_FILIAL", 
                           when(col("MATRIZ_FILIAL").isin(list(mapping_tipo.keys())), col("MATRIZ_FILIAL")).otherwise(None))
        
        for key, value in mapping_tipo.items():
            df = df.withColumn("NM_MATRIZ_FILIAL", 
                               when(col("MATRIZ_FILIAL") == key, value).otherwise(col("NM_MATRIZ_FILIAL")))
        
        # Criando a coluna ENDERECO_COMPLETO
        df = df.withColumn("ENDERECO_COMPLETO",
                           concat_ws(", ",
                                     concat(col("TIPO_LOUGRADOURO"), lit(" "), col("LOGRADOURO")),
                                     col("NUMERO"),
                                     concat_ws(" - ", col("MUNICIPIO"), col("UF"))))
        
        # Adicionando a lógica de geocodificação
        df = df.withColumn("COORDENADAS", ReceitaLT.geocode_udf(col("ENDERECO_COMPLETO")))
        
        # Caso a geocodificação falhe, tentar com o CEP
        df = df.withColumn("COORDENADAS",
                           when((col("COORDENADAS.latitude").isNull()) & (col("COORDENADAS.longitude").isNull()),
                                ReceitaLT.geocode_udf(col("CEP"))).otherwise(col("COORDENADAS")))

        # Caso a geocodificação falhe com CEP
        # df = df.withColumn("COORDENADAS",
        #                   when((col("COORDENADAS.latitude").isNull()) & (col("COORDENADAS.longitude").isNull()),
        #                        ReceitaLT.geocode_udf(col("MUNICIPIO"))).otherwise(col("COORDENADAS")))
        
        # Correção da coluna PROVEDOR
        df = df.replace(self.dic_provedor, subset=['PROVEDOR'])
        
        # Transformação das keys e values do dicionário em lowercase
        dic_prov_lower = {k.lower(): str(v).lower() for k, v in self.dic_provedor.items()}
        
        # Correção dos provedores na coluna EMAIL
        replace_expr = reduce(
            lambda a, b: regexp_replace(a, rf"\b{b[0]}\b", b[1]),
            dic_prov_lower.items(),
            col("valid_email")
        )
        
        df = df.withColumn("valid_email", replace_expr) \
               .withColumnRenamed("valid_email", "VALID_EMAIL")
        
        return df

    def process_empresas(self, df):
        """
        Processa e enriquece o DataFrame de empresas com informações adicionais e transformações.

        Parâmetros:
            df (DataFrame): DataFrame inicial contendo informações de empresas.

        Retorna:
            DataFrame: DataFrame processado e enriquecido com novas colunas e informações.

        Descrição:
            - Lê dataframes adicionais relacionados a naturezas jurídicas e qualificações.
            - Realiza renomeação de colunas para facilitar junções.
            - Enriquece o dataframe com informações de naturezas jurídicas e qualificações.
            - Processa a coluna 'NOME_EMPRESA' para extrair informações potenciais de CPF.
            - Deriva uma nova coluna baseada no porte da empresa, usando um mapeamento predefinido.
            - Determina a probabilidade de um valor ser um CPF válido com base em seu comprimento.
            - Criptografa possíveis valores de CPF usando AES e os armazena em uma nova coluna 'CPF_CRIPTOGRAFADO', enquanto remove a coluna original 'CPF'.

        Notas:
            - Esta função faz uso intensivo das operações de DataFrame do PySpark.
            - O valor de criptografia (secret_key) é gerado dinamicamente a cada chamada da função. Portanto, cada execução resultará em valores de 'CPF_CRIPTOGRAFADO' diferentes para os mesmos CPFs.
            - O método AES usado aqui é 'ECB', que não é considerado seguro para muitos casos de uso devido à falta de vetor de inicialização (IV). A utilização deste modo deve ser revista se a segurança for uma preocupação.
        """
        # Leitura dos DataFrames auxiliares
        df_nat = self.read_data(schema_name='naturezas')
        df_qual = self.read_data(schema_name='qualificacoes')
        
        # Renomear coluna para facilitar a junção
        df = df.withColumnRenamed("QUALIF_RESPONVAVEL", "COD_QUALIFICACAO")
        
        # Realizar junções com os DataFrames auxiliares
        df = df.join(broadcast(df_nat), "COD_NAT_JURICA", "left").drop("COD_NAT_JURICA")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO")
               
        # Extração e tratamento da coluna CPF
        df = df.withColumn("CPF", regexp_replace(col("NOME_EMPRESA"), "[^0-9]", ""))
        df = df.withColumn("CPF", when(col("CPF") == "", None).otherwise(col("CPF")))
        df = df.withColumn('CPF_LEN', length('CPF'))
        
        # Mapeamento para Porte da Empresa
        mapping_porte = {
            0: 'NÃO INFORMADO',
            1: 'MICRO EMPRESA',
            3: 'EMPRESA DE PEQUENO PORTE',
            5: 'DEMAIS',
            8: 'BAIXADA'
        }

        # Criando a coluna NM_PORTE
        df = df.withColumn("NM_PORTE", 
                           when(col("PORTE").isin(list(mapping_porte.keys())), col("PORTE")).otherwise(None))
        
        for key, value in mapping_porte.items():
            df = df.withColumn("NM_PORTE", 
                               when(col("PORTE") == key, value).otherwise(col("NM_PORTE")))
        
        # Determinando a probabilidade de ser um CPF válido
        df = df.withColumn("PROBABILIDADE_DE_SER_CPF", 
                           when(col("CPF_LEN") == 11, "SIM").otherwise("NAO"))
        
        # Gerando uma chave secreta para criptografia
        secret_key = secrets.token_urlsafe(24)
        df = df.withColumn("CPF_CRIPTOGRAFADO", 
                           expr(f"base64(aes_encrypt(CPF, '{secret_key}', 'ECB', 'PKCS'))")) \
               .drop("CPF")
        
        return df

    def process_simples(self, df):
        """
        Processa o DataFrame relacionado ao regime tributário SIMPLES das empresas.

        Parâmetros:
            df (pyspark.sql.DataFrame): DataFrame inicial contendo informações relacionadas ao regime tributário SIMPLES.

        Retorna:
            pyspark.sql.DataFrame: DataFrame processado com colunas de data convertidas e apenas as colunas relevantes selecionadas.

        Descrição:
            - Converte colunas que representam datas do formato "yyyyMMdd" para o tipo data.
            - Seleciona apenas as colunas relevantes para o contexto, que são: 'CNPJ_BASICO', 'OPÇAO_PELO_MEI', 'DT_OPCAO_MEI', 'DT_EXCLUSAO_MEI', 'OPCAO_PELO_SIMPLES', 'DT_OPCAO_SIMPLES', e 'DT_EXCLUSAO_SIMPLES'.

        Notas:
            - Esta função assume que as colunas de data estão no formato "yyyyMMdd" e realiza a conversão para o tipo data.
            - As colunas de datas que são processadas incluem: DATA_OPCAO_PELO_SIMPLES, DATA_EXCLUSAO_SIMPLES, DATA_EXCLUSAO_MEI e DATA_OPCAO_PELO_MEI.
        """
        # Convertendo colunas de data
        df = df.withColumn("DT_OPCAO_SIMPLES", F.to_date(col('DATA_OPCAO_PELO_SIMPLES'), "yyyyMMdd")) \
               .withColumn("DT_EXCLUSAO_SIMPLES", F.to_date(col('DATA_EXCLUSAO_SIMPLES'), "yyyyMMdd")) \
               .withColumn("DT_EXCLUSAO_MEI", F.to_date(col('DATA_EXCLUSAO_MEI'), "yyyyMMdd")) \
               .withColumn("DT_OPCAO_MEI", F.to_date(col('DATA_OPCAO_PELO_MEI'), "yyyyMMdd")) \
               .select('CNPJ_BASICO','OPÇAO_PELO_MEI','DT_OPCAO_MEI','DT_EXCLUSAO_MEI',
                       'OPCAO_PELO_SIMPLES','DT_OPCAO_SIMPLES','DT_EXCLUSAO_SIMPLES')
        
        return df

    def process_mei(self, df, save_base_path="./output/nomes", file_name="nomes.csv"):
        """
        Processa um DataFrame referente a MEIs, realiza joins com dados adicionais de naturezas jurídicas,
        qualificações, e um conjunto de dados de nomes para extrair e categorizar o primeiro nome.

        Parâmetros:
            df (pyspark.sql.DataFrame): DataFrame inicial contendo dados sobre MEIs.
            save_base_path (str, opcional): Caminho onde o arquivo com dados de nomes foi extraído. Padrão é './output/nomes'.
            file_name (str, opcional): Nome do arquivo CSV contendo dados de nomes a ser lido. Padrão é 'nomes.csv'.

        Retorna:
            pyspark.sql.DataFrame: DataFrame processado após todas as transformações e joins.

        Descrição:
            1. Realiza join com DataFrames de 'naturezas' e 'qualificações'.
            2. Extração e manipulação de dados de CPF.
            3. Utiliza um dicionário para mapear e criar a coluna "NM_PORTE".
            4. Criptografa a coluna de CPF.
            5. Realiza filtragens baseado na probabilidade do nome ser um CPF válido.
            6. Lê um conjunto de dados de nomes e realiza o explode na coluna 'alternative_names'.
            7. Extrai o primeiro nome da coluna 'NOME_EMPRESA'.
            8. Realiza o join com o conjunto de dados de nomes para categorizar o primeiro nome.
            9. Retorna um DataFrame contendo informações relevantes após todas as transformações.
        """
        # Leitura dos DataFrames auxiliares
        df_nat = self.read_data(schema_name='naturezas')
        df_qual = self.read_data(schema_name='qualificacoes')
        
        # Renomear coluna para facilitar a junção
        df = df.withColumnRenamed("QUALIF_RESPONVAVEL", "COD_QUALIFICACAO")
        
        # Realizar junções com os DataFrames auxiliares
        df = df.join(broadcast(df_nat), "COD_NAT_JURICA", "left").drop("COD_NAT_JURICA")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO")
               
        # Extração e tratamento da coluna CPF
        df = df.withColumn("CPF", regexp_replace(col("NOME_EMPRESA"), "[^0-9]", ""))
        df = df.withColumn("CPF", when(col("CPF") == "", None).otherwise(col("CPF")))
        df = df.withColumn('CPF_LEN', length('CPF'))
        
        # Mapeamento para Porte da Empresa
        mapping_porte = {
            0: 'NÃO INFORMADO',
            1: 'MICRO EMPRESA',
            3: 'EMPRESA DE PEQUENO PORTE',
            5: 'DEMAIS',
            8: 'BAIXADA'
        }

        # Criando a coluna NM_PORTE
        df = df.withColumn("NM_PORTE", 
                           when(col("PORTE").isin(list(mapping_porte.keys())), col("PORTE")).otherwise(None))
        
        for key, value in mapping_porte.items():
            df = df.withColumn("NM_PORTE", 
                               when(col("PORTE") == key, value).otherwise(col("NM_PORTE")))
        
        # Determinando a probabilidade de ser um CPF válido
        df = df.withColumn("PROBABILIDADE_DE_SER_CPF", 
                           when(col("CPF_LEN") == 11, "SIM").otherwise("NAO"))
        
        # Gerando uma chave secreta para criptografia
        secret_key = secrets.token_urlsafe(24)
        df = df.withColumn("CPF_CRIPTOGRAFADO", 
                           expr(f"base64(aes_encrypt(CPF, '{secret_key}', 'ECB', 'PKCS'))")) \
               .drop("CPF")
        
        # Filtrar df_processado baseado na coluna PROBABILIDADE_DE_SER_CPF
        df_filter = df.filter(col('PROBABILIDADE_DE_SER_CPF') == 'SIM') \
                      .dropDuplicates(subset=['CPF_CRIPTOGRAFADO', 'NOME_EMPRESA'])
        
        # Caminho completo do arquivo de nomes
        file_path = os.path.join(save_base_path, file_name)
        
        # Ler o arquivo CSV de nomes
        df_csv = self.spark.read.csv(file_path, header=True, inferSchema=True)
        
        # Explodir a coluna alternative_names para múltiplas linhas
        df_csv = df_csv.withColumn("alternative_names", explode(split(coalesce(col("alternative_names"), col("first_name")), "\\|")))
        
        # Selecionar as colunas desejadas
        df_result = df_csv.select("alternative_names", "group_name", "ratio", "classification") \
                          .dropDuplicates(subset=['alternative_names'])
        
        # Extrair o primeiro nome da coluna NOME_EMPRESA
        df_filter = df_filter.withColumn("PRIMEIRO_NOME", split(col("NOME_EMPRESA"), " ")[0])
        
        # Fazer o join entre df_filter e df_result
        joined_df = df_filter.join(df_result, df_filter.PRIMEIRO_NOME == df_result.alternative_names, "left").dropDuplicates()
        
        # Selecionar as colunas finais
        joined_df = joined_df.select(
            'CNPJ', 'NOME_EMPRESA', 'CAP_SOCIAL', 'NM_PORTE', 'NAT_JURICA', 
            'ENTE_FEDERATIVO', 'NM_QUALIFICACAO', 'CPF_CRIPTOGRAFADO', 'CPF_LEN',
            'PROBABILIDADE_DE_SER_CPF', 'PRIMEIRO_NOME',
            col('group_name').alias('GRUPO_NOME'), 
            col('ratio').alias('PROBABILIDADE_CLASSIFICACAO'), 
            col('classification').alias('CLASSIFICACAO')
        ).dropDuplicates()
        
        return joined_df

    def process_socios(self, df, save_base_path="./output/nomes", file_name="nomes.csv"):
        """
        Processa um DataFrame referente a sócios, realiza joins com dados adicionais de países, qualificações, 
        e um conjunto de dados de nomes para extrair e categorizar o primeiro nome.

        Parâmetros:
            df (pyspark.sql.DataFrame): DataFrame inicial contendo dados sobre sócios.
            save_base_path (str, opcional): Caminho onde o arquivo com dados de nomes foi extraído. Padrão é './output/nomes'.
            file_name (str, opcional): Nome do arquivo CSV contendo dados de nomes a ser lido. Padrão é 'nomes.csv'.

        Retorna:
            pyspark.sql.DataFrame: DataFrame processado após todas as transformações e joins.

        Descrição:
            1. Realiza join com DataFrames de 'países'.
            2. Usa mapeamentos para criar colunas "NM_FAIXA_ETARIA" e "NM_IDENTIFICADOR_SOCIO".
            3. Renomeia e realiza join com DataFrame de qualificações para obter descrições das qualificações.
            4. Converte coluna de data "DATA_ENTRADA_SOCIEDADE" para o formato desejado.
            5. Lê um conjunto de dados de nomes e realiza o explode na coluna 'alternative_names'.
            6. Extrai o primeiro nome da coluna 'NOME_SOCIO_RAZAO_SOCIAL'.
            7. Realiza o join entre o DataFrame processado e o conjunto de dados de nomes para categorizar o primeiro nome.
            8. Retorna um DataFrame contendo informações relevantes após todas as transformações.
        """
        # Leitura dos DataFrames auxiliares
        df_pais = self.read_data(schema_name='paises')
        df = df.withColumnRenamed("PAIS", "COD_PAIS")
        df = df.join(broadcast(df_pais), "COD_PAIS", "left").drop("COD_PAIS")
        
        # Mapeamento de códigos para faixas etárias.
        mapping_faixa_etaria = {
            1: '0 a 12 anos',
            2: '13 a 20 anos',
            3: '21 a 30 anos',
            4: '31 a 40 anos',
            5: '41 a 50 anos',
            6: '51 a 60 anos',
            7: '61 a 70 anos',
            8: '71 a 80 anos',
            9: 'maiores de 80 anos',
            0: 'NA'
        }
        
        # Mapeamento de códigos para identificador de sócio.
        mapping_id_socio = {
            1: 'PESSOA JURIDICA',
            2: 'PESSOA FISICA',
            3: 'ESTRANGEIRO'
        }
        
        # Leitura do DataFrame de qualificações
        df_qual = self.read_data(schema_name='qualificacoes')
        
        # Criando a coluna NM_FAIXA_ETARIA
        df = df.withColumn("NM_FAIXA_ETARIA", 
                           when(col("FAIXA_ETARIA").isin(list(mapping_faixa_etaria.keys())), col("FAIXA_ETARIA")).otherwise(None))
        
        for key, value in mapping_faixa_etaria.items():
            df = df.withColumn("NM_FAIXA_ETARIA", 
                               when(col("FAIXA_ETARIA") == key, value).otherwise(col("NM_FAIXA_ETARIA")))
        
        # Criando a coluna NM_IDENTIFICADOR_SOCIO
        df = df.withColumn("NM_IDENTIFICADOR_SOCIO", 
                           when(col("IDENTIFICADOR_SOCIO").isin(list(mapping_id_socio.keys())), col("IDENTIFICADOR_SOCIO")).otherwise(None))
        
        for key, value in mapping_id_socio.items():
            df = df.withColumn("NM_IDENTIFICADOR_SOCIO", 
                               when(col("IDENTIFICADOR_SOCIO") == key, value).otherwise(col("NM_IDENTIFICADOR_SOCIO")))
        
        # Renomear e juntar com qualificações para obter descrições
        df = df.withColumnRenamed("QUALIFICACAO_REPRESENTANTE_LEGAL", "COD_QUALIFICACAO")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO") \
               .withColumnRenamed("NM_QUALIFICACAO", "NM_QUALIFICACAO_REPRESENTANTE_LEGAL")
        
        df = df.withColumnRenamed("QUALIFICAÇAO_SOCIO", "COD_QUALIFICACAO")
        df = df.join(broadcast(df_qual), "COD_QUALIFICACAO", "left").drop("COD_QUALIFICACAO") \
               .withColumnRenamed("NM_QUALIFICACAO", "NM_QUALIFICAÇAO_SOCIO")
        
        # Conversão da coluna de data
        df = df.withColumn("DT_ENTRADA_SOCIEDADE", F.to_date(col('DATA_ENTRADA_SOCIEDADE'), "yyyyMMdd")) \
               .drop("DATA_ENTRADA_SOCIEDADE")
        
        # Leitura do arquivo CSV de nomes
        file_path = os.path.join(save_base_path, file_name)
        df_csv = self.spark.read.csv(file_path, header=True, inferSchema=True)
        
        # Explodir a coluna alternative_names para múltiplas linhas
        df_csv = df_csv.withColumn("alternative_name2", explode(split(col("alternative_names"), "\|")))
        
        # Selecionar as colunas desejadas
        df_result = df_csv.select("alternative_name2", "group_name", "ratio", "classification") \
                          .dropDuplicates(["alternative_name2"])
        
        # Extrair o primeiro nome da coluna NOME_SOCIO_RAZAO_SOCIAL
        df = df.withColumn("PRIMEIRO_NOME", split(col("NOME_SOCIO_RAZAO_SOCIAL"), " ")[0])
        
        # Fazer o join entre df_filter e df_result
        joined_df = df.join(df_result, df.PRIMEIRO_NOME == df_result.alternative_name2, "left").dropDuplicates()
        
        # Selecionar as colunas finais
        joined_df = joined_df.select(
            'CNPJ_BASICO', 'NOME_SOCIO_RAZAO_SOCIAL', 'CNPJ_CPF_SOCIO', 'REPRESENTANTE_LEGAL',
            'NOME_REPRESENTANTE', 'NM_PAIS', 'NM_FAIXA_ETARIA', 'NM_IDENTIFICADOR_SOCIO',
            'NM_QUALIFICACAO_REPRESENTANTE_LEGAL', 'NM_QUALIFICAÇAO_SOCIO', 'DT_ENTRADA_SOCIEDADE',
            'PRIMEIRO_NOME', 
            col('ratio').alias('PROBABILIDADE_CLASSIFICACAO'),
            col('classification').alias('CLASSIFICACAO')
        ).dropDuplicates()
        
        return joined_df

In [10]:
#Baixar o arquivo de nomes para realizar tratamento de sexo nas tabelas MEI e Socios
#receitaLT_processor = ReceitaLT(spark)
#receitaLT_processor.download_nomes()

In [11]:
receitaLT_processor = ReceitaLT(spark)
df = receitaLT_processor.read_data(schema_name='estabelecimentos')
df_estabelecimentos = receitaLT_processor.process_estabelecimentos(df)
df_estabelecimentos = df_estabelecimentos.select('CNPJ_BASICO','CNPJ_ORDEM', 'NM_MATRIZ_FILIAL', 'NM_SIT_CADASTRAL', 'DT_SIT_CADASTRAL',
    'NOME_FANTASIA', 'DT_INICIO_ATIVIDADE', 'ENDERECO_COMPLETO', 'TIPO_LOUGRADOURO', 'CEP', 'UF', 'COORDENADAS', 'MUNICIPIO',
    'DDD1','TEL1', 'DDD2','TEL2', 'VALID_EMAIL', 'PROVEDOR', 'NM_MOTIVO', 'CNAE', 'NM_PAIS', 'ano_cadastro', 'mes_cadastro', 'ano_sit_cadastral','mes_sit_cadastral').cache()
df_estabelecimentos.repartition(25).write.mode("overwrite").parquet('C:/Users/pedro/Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_estab/')

INFO:__main__:Detected encodings: {'./output\\Estabelecimentos\\Estabelecimentos.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos0.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos1.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos2.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos3.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos4.csv': 'ISO-8859-1', './output\\Estabelecimentos\\Estabelecimentos5.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos6.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos7.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos8.csv': 'ISO-8859-1', './output\\Estabelecimentos\\Estabelecimentos9.csv': 'ISO-8859-1'}
INFO:__main__:Arquivo lido com sucesso: ./output\Estabelecimentos\Estabelecimentos.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Estabelecimentos\Estabelecimentos0.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Estabelecimentos\Estabelecimentos1.csv
INFO:_

In [15]:
receitaLT_processor = ReceitaLT(spark)
df = receitaLT_processor.read_data(schema_name='estabelecimentos')
df_estabelecimentos = receitaLT_processor.process_estabelecimentos(df)
df_estabelecimentos = df_estabelecimentos.select('CNPJ_BASICO','CNPJ_ORDEM', 'NM_MATRIZ_FILIAL', 'NM_SIT_CADASTRAL', 'DT_SIT_CADASTRAL',
    'NOME_FANTASIA', 'DT_INICIO_ATIVIDADE', 'ENDERECO_COMPLETO', 'TIPO_LOUGRADOURO', 'CEP', 'UF', 'COORDENADAS', 'MUNICIPIO',
    'DDD1','TEL1', 'DDD2','TEL2', 'VALID_EMAIL', 'PROVEDOR', 'NM_MOTIVO', 'CNAE', 'NM_PAIS', 'ano_cadastro', 'mes_cadastro', 'ano_sit_cadastral','mes_sit_cadastral').cache()

INFO:__main__:Detected encodings: {'./output\\Estabelecimentos\\Estabelecimentos.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos0.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos1.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos2.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos3.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos4.csv': 'ISO-8859-1', './output\\Estabelecimentos\\Estabelecimentos5.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos6.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos7.csv': 'ascii', './output\\Estabelecimentos\\Estabelecimentos8.csv': 'ISO-8859-1', './output\\Estabelecimentos\\Estabelecimentos9.csv': 'ISO-8859-1'}
INFO:__main__:Arquivo lido com sucesso: ./output\Estabelecimentos\Estabelecimentos.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Estabelecimentos\Estabelecimentos0.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Estabelecimentos\Estabelecimentos1.csv
INFO:_

In [ ]:
df_estabelecimentos

In [12]:
df_estabelecimentos = df_estabelecimentos.select('CNPJ_BASICO','CNPJ_ORDEM', 'NM_MATRIZ_FILIAL', 'NM_SIT_CADASTRAL', 'DT_SIT_CADASTRAL',
    'NOME_FANTASIA', 'DT_INICIO_ATIVIDADE', 'ENDERECO_COMPLETO', 'TIPO_LOUGRADOURO', 'CEP', 'UF', 'COORDENADAS', 'MUNICIPIO',
    'DDD1','TEL1', 'DDD2','TEL2', 'VALID_EMAIL', 'PROVEDOR', 'NM_MOTIVO', 'CNAE', 'NM_PAIS', 'ano_cadastro', 'mes_cadastro', 'ano_sit_cadastral','mes_sit_cadastral').cache()

In [ ]:
df_estabelecimentos.repartition(25).write.mode("overwrite").parquet('C:/Users/pedro/Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_estab/')

In [ ]:
Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_estab

In [13]:
receitaLT_processor = ReceitaLT(spark) 
dfmei = receitaLT_processor.read_data(schema_name='empresas')
df_mei = receitaLT_processor.process_mei(dfmei)

INFO:__main__:Detected encodings: {'./output\\Empresas\\Empresas0.csv': 'ascii', './output\\Empresas\\Empresas1.csv': 'ascii', './output\\Empresas\\Empresas2.csv': 'ascii', './output\\Empresas\\Empresas3.csv': 'ascii', './output\\Empresas\\Empresas4.csv': 'ascii', './output\\Empresas\\Empresas5.csv': 'ascii', './output\\Empresas\\Empresas6.csv': 'ascii', './output\\Empresas\\Empresas7.csv': 'ascii', './output\\Empresas\\Empresas8.csv': 'ascii', './output\\Empresas\\Empresas9.csv': 'ascii'}
INFO:__main__:Arquivo lido com sucesso: ./output\Empresas\Empresas0.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Empresas\Empresas1.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Empresas\Empresas2.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Empresas\Empresas3.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Empresas\Empresas4.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Empresas\Empresas5.csv
INFO:__main__:Arquivo lido com sucesso: ./output\Empresas\Empresas6.csv
I

In [14]:
df_mei = df_mei.withColumn('CLASSIFICACAO', when((df_mei.PRIMEIRO_NOME == 'JOSE') & 
                                         (df_mei.CLASSIFICACAO == 'F'), 'M').otherwise(df_mei.CLASSIFICACAO))
df_mei = df_mei.select('CNPJ','NOME_EMPRESA','CAP_SOCIAL', 'NAT_JURICA', 'NM_QUALIFICACAO', 'CLASSIFICACAO', 'PROBABILIDADE_CLASSIFICACAO', ).cache()
df_mei.repartition(20).write.mode("overwrite").parquet('C:/Users/pedro/Documents/Curso de pos graduação de EST/DADOS_CNPJ/df_mei/')

In [15]:
df_mei

CNPJ,NOME_EMPRESA,CAP_SOCIAL,NM_PORTE,NAT_JURICA,ENTE_FEDERATIVO,NM_QUALIFICACAO,CPF_CRIPTOGRAFADO,CPF_LEN,PROBABILIDADE_DE_SER_CPF,PRIMEIRO_NOME,GRUPO_NOME,PROBABILIDADE_CLASSIFICACAO,CLASSIFICACAO
34848565,KARINA BELARMINO ...,"1000,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,+++I1S0KUamFe+pHK...,11,SIM,KARINA,KARINA,0.9961922744079931,F
28191842,ROSEMEIRE PERES R...,"4000,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,+++OOoMxkfHjcQ8Gm...,11,SIM,ROSEMEIRE,ROSEMEIRE,1.0,F
45631914,ARTUR LUIS PEREIR...,"300,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,+++uvDpUzPWjKtub/...,11,SIM,ARTUR,ARTUR,0.988292948543425,M
15595525,GILMAR COSTA DA S...,"1,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,++/hF69NvCTMiYLZz...,11,SIM,GILMAR,GILMAR,1.0,F
43695636,GABRIELA MARTINS ...,"5000,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,++/y1+lNL/wqrRn56...,11,SIM,GABRIELA,GABRIELA,1.0,F
12279870,MARIA APPARECIDA ...,"1,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,++09S+KdUALSSk61I...,11,SIM,MARIA,MARIA,1.0,F
17158845,FERNANDA DE SOUZA...,"5000,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,++18KWzBzrMCR94pA...,11,SIM,FERNANDA,FERNANDA,1.0,F
19872585,KELY ACOSTA ARIM ...,"100,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,++1IiPpha6RhYEkfb...,11,SIM,KELY,KELI,0.5982142857142857,F
12071862,LUIZ ALBERTO OLIV...,"1,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,++29WNrhM2ZfWI1Vg...,11,SIM,LUIZ,LUIZ,1.0,M
42768909,WAYANY COUTINHO B...,"5000,00",MICRO EMPRESA,Empresário (Indiv...,NULL,Empresário,++2xDlWYsZrRQPH2v...,11,SIM,WAYANY,NULL,NULL,NULL


In [15]:
df_final = df_mei.join(df_estabelecimentos, df_mei.CNPJ==df_estabelecimentos.CNPJ_BASICO, 'inner')

In [ ]:
df_final

In [ ]:
df_final.count()

In [1]:
    def detect_encoding(file_pattern_or_path, num_bytes=10000):
        """
        Detecta a codificação do arquivo ou arquivos fornecidos.
        
        Parâmetros:
            file_pattern_or_path (str): Caminho ou padrão do arquivo para detecção.
            num_bytes (int, opcional): Número de bytes para ler para a detecção. Padrão é 10000.
        
        Retorna:
            dict: Dicionário com caminho do arquivo como chave e codificação detectada como valor.
        """
        files = glob.glob(file_pattern_or_path)
        encodings = {}
        for file_path in files:
            with open(file_path, 'rb') as f:
                rawdata = f.read(num_bytes)
                encodings[file_path] = chardet.detect(rawdata)["encoding"]
        return encodings

In [ ]:
receitaLT_processor = ReceitaLT(spark)
df = receitaLT_processor.read_data(schema_name='socios')
df_socios = receitaLT_processor.process_socios(df).cache()

In [ ]:
df_socios

In [ ]:
receitaLT_processor = ReceitaLT(spark)
df = receitaLT_processor.read_data(schema_name='simples')
df_simples = receitaLT_processor.process_simples(df).cache()

In [ ]:
df_simples

In [ ]:
receitaLT_processor = ReceitaLT(spark) 
df = receitaLT_processor.read_data(schema_name='empresas')
df_empresas = receitaLT_processor.process_empresas(df).cache()